<h1 style="text-align:Center; color:orange;">- Brooks Tools - Microbiome Analysis Pipeline -</h1>
<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h4 style="text-align:center; color:blue;">Andrew W. Brooks</h4>
<h4 style="text-align:center; color:blue;">Vanderbilt Genetics Institute</h4>
<h4 style="text-align:center; color:blue;">andrew.w.brooks(at)vanderbilt.edu</h4>
<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h4 style="text-align:center; color:black;">Released under MIT License</h4>
<h4 style="text-align:center; color:black;">Copyright (c) 2017 Andrew W. Brooks</h4>
<h4 style="text-align:center; color:black;">Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. The software is provided "as is", without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement. In no event shall the authors or copyright holders be liable for any claim, damages or other liability, whether in an action of contract, tort or otherwise, arising from, out of or in connection with the software or the use or other dealings in the software.</h4>
<h4 style="text-align:center; color:red;"></h4>
<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>

<h1 style="text-align:center; color:orange;"> - Prepare Toolkit - </h1>

<h4 style="text-align:center; color:blue;"> - Load Libraries and Environment - </h4>

In [1]:
#!/usr/bin/env python
__author__ = "Andrew Brooks"
__version__ = "1.0"
__license__ = "MIT"
__copyright__ = "Andrew Brooks"
__email__ = "andrew.w.brooks@vanderbilt.edu"
__status__ = "Alpha"

##### BROOKS UTILITIES #####################################################

##### LOAD PACKAGES #####
### GENERAL UTILITIES ###
import glob         # TO REGEX SEARCH FOR FILES 
import os           # TO MAKE COMMAND LINE CALLS ()
import random       # TO GENERATE RANDOM VALUES
import copy         # TO COPY COMPLICATED OBJECTS

### DATA STRUCTURES ###
import pandas as pd # PANDAS DATAFRAMES
from IPython.display import display # USE TO DISPLAY PANDAS
import numpy as np  # NUMPY NUMERICAL AND LINEAR ALGEBRA TOOLKIT
random.seed(54321)  # SET RANDOM SEED FOR REPRODUCIBILITY #

### DATA PROCESSING
import scipy as sp  # SCIPY SCIENTIFIC TOOLKIT
from scipy.cluster.hierarchy import linkage # UPGMA CLUSTER TOOL
import skbio as sk  # SCIKIT-BIO TOOLS FOR BIOM ANALYSES
from skbio.stats.distance import DistanceMatrix # DISTANCE MATRIX OBJECT FOR BETA DIVERSITY
from skbio.tree import TreeNode # BACTERIAL PHYLOGENY OBJECT
import itertools # ACCESS DATA ITERATION COMBINATIONS
from statsmodels.sandbox.stats.multicomp import multipletests # PERFORM FDR P-VALUE CORRECTION
import statsmodels.api as sm
from decimal import Decimal
import patsy # FORMAT DATA FOR REGRESSION ANALYSES 

### BIOM JSON OBJECTS ###
from biom.table import Table # BIOM TABLE COMPLEX DATA STRUCTURE
from biom import load_table  # LOAD BIOM TABLE FROM FILE (.BIOM)

### PLOTTING ###
%matplotlib inline
import matplotlib.pyplot as plt # MATPLOTLIB PLOTTING TOOLS
from matplotlib.backends.backend_pdf import PdfPages # MATPLOTLIB SAVE MULTIPAGE PDFS
import seaborn as sns # SEABORN PLOTTING TOOLS (FRIENDLY WITH PANDAS)
from emperor import Emperor # EMPEROR PCOA PLOTING

import warnings


<h4 style="text-align:center; color:blue;"> - Load Basic Tools - </h4>

In [2]:
##### BASIC TOOLS - I/O ###################################################################

##### LIST #####
### PRINT A LIST LINE BY LINE TO CONSOLE ###
def list_print(listIn): 
    for listValIn in listIn: print(listValIn)
### WRITE A LIST LINE BY LINE TO FILE (OVERWRITES FILE) ###
def list_write(listIn, filePathIn):
    fOut = open(filePathIn, 'w')
    for listValIn in listIn:
        fOut.write((listValIn+"\n"))
    fOut.close()
    
### WRITE A LIST LINE BY LINE TO FILE (APPENDS TO FILE) ###
def list_append(listIn, filePathIn, addBlankTail=True):
    fOut = open(filePathIn, 'a')
    for listValIn in listIn:
        fOut.write((listValIn+"\n"))
    if addBlankTail == True: 
        fOut.write("\n")
    fOut.close()
    
### TAKE IN LIST OF STRINGS AND CONVERT TO SINGLE STRING SEPARATED BY \r ###
def list_to_string(listIn):
    strOut = ""
    for listCycle in listIn: strOut = strOut + listCycle + "\r"
    return strOut

<h4 style="text-align:center; color:blue;"> - Load Statistics Tools - </h4>

In [3]:
##### STATISTICS - FREQUENTIST ############################################################
    
##### MANN WHITNEY U TEST ##### list of results
def stats_mannwhitney(l1, l2, l1Name=None, l2Name=None, bonferroniComparisons=1):
    # use_continuity = Whether a continuity correction (1/2.) should be taken into account. Default is True. [Scipy]
    outMann = sp.stats.mannwhitneyu(l1, l2, use_continuity=True, alternative='two-sided')
    outStrList = ["Mann Whitney U - Nonparametric Rank Test"]
    if (l1Name != None) & (l2Name != None): outStrList.append("    Comparing: "+l1Name+" with "+l2Name)
    outStrList.append("    List #1 Length: "+str(len(l1))+" | List #2 Length: "+str(len(l2)))
    outStrList.append("    Test Statistic: "+str(outMann[0]))
    # PRINT P-VALUES FOR ONE AND TWO TAILED
    outStrList.append("    P-Value (onetailed): "+str(outMann[1]/2))
    outStrList.append("    P-Value (twotailed): "+str(outMann[1]))
    # IF BONFERRONI CORRECTION FOR MORE THAN ONE TEST: CORRECT AND CHECK IF LARGER THAN 1
    if (bonferroniComparisons != 1) and (bonferroniComparisons != None):
        if (outMann[1]/2)*bonferroniComparisons <= 1: outStrList.append("    P-Value Bonferroni Corrected for "+str(bonferroniComparisons)+" tests: "+str((outMann[1]/2)*bonferroniComparisons))
        else: outStrList.append("    P-Value Bonferroni Corrected for "+str(bonferroniComparisons)+" tests: "+str(1.0))
        if ((outMann[1])*bonferroniComparisons) <= 1: outStrList.append("    P-Value Bonferroni Corrected for "+str(bonferroniComparisons)+" tests (twotailed): "+str(outMann[1]*bonferroniComparisons))
        else: outStrList.append("    P-Value Bonferroni Corrected for "+str(bonferroniComparisons)+" tests (twotailed): "+str(1.0))
    return outStrList, outMann[0], outMann[1]
                    

### TAKE IN LIST OF P-VALUES AND RETURN DATAFRAME OF P-VALUES & FDR &  BONFERRONI CORRECTED P-VALUES ###
def stats_pcorrect(pValList,alphaIn=0.05):
    ### GET FDR P-VALUES ###
    fdrResIn = multipletests(pValList, alpha=alphaIn, method='fdr_bh')
    fdrPVals = fdrResIn[1]
    ### GET BONFERRONI P-VALUES ###
    bonferroniPVals = np.multiply(pValList, len(pValList))
    ### FORMAT RESULTS AS DATAFRAME ###
    dfResults = pd.DataFrame({'p-raw':pValList, 'p-fdr':fdrPVals, 'p-bonferroni':bonferroniPVals})
    ### REMOVE BONFERRONI P-VALUES > 1 ###
    dfResults[dfResults['p-bonferroni'] > 1] = 1.0
    ### RETURN DATAFRAME OF RAW, FDR, AND BONFERRONI CORRECTED P-VALUES ###
    return dfResults[['p-raw','p-fdr','p-bonferroni']]  

### LINEAR REGRESSION - ORDINARY LEAST SQUARES
# regResults, regDependent, regPredictors = stats_regression(mapDF, regEquation=" age_years ~ bmi + race + race:sex ")
def stats_regression(pdDFIn, regEquation=" age_years ~ bmi + sex:race "):
    ### GET VARIABLES INTO X[n,p] predictors and y[n,1] outcome
    y, X = patsy.dmatrices(regEquation, pdDFIn, return_type='dataframe')
    ### GENERATE OLS REGRESSION MODEL ###
    statsModel = sm.OLS(y, X)
    ### FIT DATA TO A LINE USING OLS MINIMIZATION ###
    statsModelFit = statsModel.fit()
    ### PRINT RESULTS ###
    print(statsModelFit.summary())
    ### RETURN: resultsObject, y, X
    return statsModelFit, y, X

### LINEAR REGRESSION - RIDGE REGRESSION - COEFFICIENTS PENALTIES
# regResults, regDependent, regPredictors = stats_regression_ridge(mapDF, regEquation=" age_years ~ bmi + race + race:sex ", alphaIn=0.5)
def stats_regression_ridge(pdDFIn, regEquation=" age_years ~ bmi + sex:race ", alphaIn=0.5):
    ### GET VARIABLES INTO X[n,p] predictors and y[n,1] outcome
    y, X = patsy.dmatrices(regEquation, pdDFIn, return_type='dataframe')
    ### GENERATE OLS REGRESSION MODEL ###
    statsModel = sl.linear_model.Ridge(alpha = alphaIn)
    ### FIT DATA TO A LINE USING OLS MINIMIZATION ###
    statsModelFit = statsModel.fit(X,y)
    ### PRINT RESULTS ###
    ###print(statsModelFit.summary())
    
    # The coefficients
    print('Coefficients: \n', statsModelFit.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % np.mean((statsModelFit.predict(X) - y) ** 2))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % statsModelFit.score(X, y))
    ### RETURN: resultsObject, y, X
    return statsModelFit, y, X



<h4 style="text-align:center; color:blue;"> - Load Pandas Tools - </h4>

In [4]:
####### PANDAS & STATISTICAL TOOLS (DataFrame - pandas ) ####################################
# Class to contain a pandas dataframe and additional info
# Tools include: Statistics, Plotting
class DF:
    """
    ### DF PANDAS DATAFRAME TOOLKIT ###
    
    
    INPUT:
        - objIn can be:
            - str: will search as path for delimited table
            - pd.DataFrame(): will initialize as copy
            - ndarray(structured dtype), list of tuples, dict, or DataFrame
            
            
            
            - dict{key:item,key:item}: will convert
            - !!! List[of tuples (1,'a',True),(2,'b',False)]
    
    USE: 
        
    
        DF( pathIn=None, sepIn='\t', ### IMPORT FROM FILE... ###
            dictIn=None, ### ...OR IMPORT FROM DICT... ###
            dataIn=None, columnsIn=None, indexIn=None, ### ...OR BUILD FROM DATA ###
            verboseIn=True, ### WHETHER TO PRINT ###
                )
    INPUT:
       keys           : column name or list[column names]
       dropIn=True      : delete columns to become index
       appendIn=False   : add to existing 
       inplace=False  : perform on target dataframe
    RETURN: None
    TYPE: function
    STRUCTURE: DF()

    """

    ### CLASS INITIALIZATION ##################################################
    def __init__(self, objIn, columnsIn=None, indexIn=None, sepIn='\t', verboseIn=True):
        ### DATA STRUCTURES ##############################
        ##### VERBOSE BOOL .verbose #########
        self.verbose = verboseIn
        ##### SEPARATOR BOOL .sep #########- Default '\t'
        self.sep=sepIn
        ##### INPUT PATH .path #########
        self.source=objIn
        ### INDEX AND COLUMNS ###
        if columnsIn is None: columnsIn=[]
        if indexIn is None: indexIn=[]
        self.columns=columnsIn;self.c=columnsIn
        self.index=indexIn;self.i=indexIn
        self.palette = None
        
        ##### DATAFRAME .df #########
        iB=False # BOOL INPUT SUCCESS #
        if isinstance(self.source, str): self.in_tsv();iB=True ### IF STRING IMPORT AS TSV PATH ###
        elif self.source is None: ### IF NONE THEN MAKE EMPTY DATAFRAME
            self.df = pd.DataFrame(index=indexIn, columns=columnsIn, dtype=None, copy=False)
            self.df.fillna(0); iB = True
        elif (isinstance(self.source, int)) or (isinstance(self.source,float)):
            self.df = pd.DataFrame(index=indexIn, columns=columnsIn, dtype=None, copy=False)
            self.df.fillna(self.source); iB = True
        elif isinstance(self.source, pd.DataFrame): self.df = self.source.copy();iB=True ### IF DF THEN COPY ##
        elif isinstance(self.source, dict): self.in_dict(self.source);iB=True ### IF DICT THEN FORMAT AS DF ###
        else:  ###  TRY IMPORTING AS OBJECT ### 
            self.in_obj(self.source);iB=True
             
        if not iB: 
            if self.verbose is True: " - Initializing as Nan Dataframe - "
            self.df = pd.DataFrame(index=indexIn, columns=columnsIn, dtype=None, copy=False);self.df.fillna(0)
        
        ### RESET SOURCE FOR RESET FUNCTION ###
        self.source = self.copy()
        
        ##### UPDATE FROM DATAFRAME ######################
        self.update()
        
    ##########################################################################
    ########################## COPY ##########################################
    ### COPY .copy() Return unconnected copy of class object.
    def copy(self): return copy.deepcopy(self)

    ##########################################################################
    ########################## DISPLAY #######################################
    
    ### DISPLAY IN NOTEBOOK ###
    def d(self): display(self.df)
    def display(self): display(self.df)
    ### DISPLAY PANDAS - SCIENTIFIC NOTATION ###
    def display_scientific(self, dispPos=3): pd.set_option('display.float_format', '{:.'+str(dispPos)+'g}'.format)
    ### DISPLAY PANDAS - FLOAT NOTATION ###
    def display_float(self, dispPos=3): 
        pd.set_option('display.float_format', '{:.5f}'.format)
    ### DISPLAY PANDAS - MAX COLUMN WIDTH (-1 = No Wrap) ###
    def display_colwidth(self, widthIn=-1): pd.set_option('display.max_colwidth', widthIn)
 
    ##########################################################################
    ########################### I/O ##########################################    
    
    ### DF FROM TSV ### -> self
    def in_tsv(self):
        if self.verbose is True: print(" - Loading DF File - " + self.source);
        self.df = pd.read_csv(self.source, sep=self.sep, index_col=None, skiprows=0, verbose=False, low_memory=False) 
        if self.verbose is True: print(" - SUCCESS: Loaded DF from: "+self.source+" - ")
        return self
    ### DF FROM DICTIONARY ### -> self
    def in_dict(self, dictIn):
        if self.verbose is True: print(" - Creating DF from Dictionary - ")
        self.df = pd.DataFrame.from_dict(dictIn, orient='columns', dtype=None)
        if self.verbose is True: print(" - SUCCESS: Created DF from Dictionary - ")
        return self
    ### DF FROM RECORD ###  -> self (objIn i.e. ndarray(i.e.structured dtype), list of tuples, dict, or DataFrame
    def in_obj(self, objIn):
        if self.verbose is True: print(" - Creating DF from Object - "+self.columns)
        self.df = pd.DataFrame.from_records(data=objIn, columns=self.columns, index=self.index, coerce_float=True)
        if self.verbose is True: print(" - SUCCESS: Created DF from Object - ")
        return self
    ### DF TO TSV FILE ### -> self
    def out_tsv(self, outPath):
        if self.verbose is True: print(" - Writing DF to File "+outPath+" - ")
        self.df.to_csv(outPath, sep=self.sep)
        if self.verbose is True: print(" - Written DF to File "+outPath+" - ")
        return self
    
    ##########################################################################
    ############################ PALETTE #####################################
    ### DISPLAY A PREVIEW OF THE PALETTE ...ooooh pretty ###
    def palette_preview(self): sns.palplot(self.palette)
    ### SET .palette FOR A RAINBOW OF COLORS ###
    def palette_heatmap(self, numColors): self.palette = sns.palplot(sns.color_palette("coolwarm", numColors))
    ### SET .palette FOR A HEATMAP OF COLORS ###
    def palette_rainbow(self, numColors, lightIn=0.5, saturationIn=0.8, previewIn=False): self.palette = sns.hls_palette(numColors, l=lightIn, s=saturationIn)

 
    ##########################################################################
    ########################### PLOTTING #####################################
    ### LINEAR MODEL PLOT ### continuousXcontinuous, 95%CI, can color and split plots by categories.
    def plot_lm(self, continuousColX, continuousColY, categoricalColor=None, categoricalRow=None, categoricalColumn=None):
        ### SET STYLE ###
        sns.set(style="ticks", context="talk"); pal=None
        ### GET COLOR PALETTE ###
        self.palette_rainbow(len(self.df[categoricalColor].unique()), lightIn=0.7, saturationIn=1.0 )
        plt.clf();plt.title('Winning')
        ### MAKE LINEAR MODEL PLOT ###
        return sns.lmplot(x=continuousColX,y=continuousColY,row=categoricalRow,col=categoricalColumn,hue=categoricalColor,data=self.df,size=7, palette=self.palette, ci=95)    
    ### VIOLIN PLOT ### categoricalXcontinuous, color ideal with two groups.
    def plot_violin(self, categoricalColX, continuousColY, categoricalColor=None):
        ### IF CATEGORICAL COLOR ON EACH HALF OF VIOLIN ###
        splitIn = False; 
        if len(self.df[categoricalColor].unique()) == 2: splitIn = True
        ### GET COLOR PALETTE ###
        self.palette_rainbow(len(self.df[categoricalColor].unique()), lightIn=0.7, saturationIn=1.0 )
        ### MAKE VIOLIN PLOT ###
        return sns.violinplot(x=categoricalColX, y=continuousColY, hue=categoricalColor, data=self.df, split=splitIn, palette=self.palette)
        #sns.despine(left=True)
    
    ##########################################################################
    ########################## RESET #########################################
    ### RESET .reset()
    def reset(self): return self.__init__(self.source)
    
    ##########################################################################
    ########################## STATS #########################################
    
    ### REPLACE IN DATAFRAME ###
    def replace(self, toReplace, replaceWith): self.df.replace(to_replace=toReplace, value=replaceWith, inplace=True, limit=None, regex=False, method='pad')
  
    ### REPLACE STRING WITH REGEX ###
    def replace_string(self, regexToReplace, replaceWithStr): self.df.replace(to_replace=regexToReplace, value=replaceWithStr, inplace=True, limit=None, regex=True)    

    ##########################################################################
    ########################## STATS ####################################
    ##### .stats_X STATISTICS FUNCTIONS #####################
    
    ### LINEAR REGRESSION - ORDINARY LEAST SQUARES
    # regResults, regDependent, regPredictors = stats_regression(mapDF, regEquation=" age_years ~ bmi + race + race:sex ")
    def stats_regression(self, regEquation=" age_years ~ bmi + sex:race "):
        ### GET VARIABLES INTO X[n,p] predictors and y[n,1] outcome
        y, X = patsy.dmatrices(regEquation, self.df, return_type='dataframe')
        ### GENERATE OLS REGRESSION MODEL ###
        statsModel = sm.OLS(y, X)
        ### FIT DATA TO A LINE USING OLS MINIMIZATION ###
        statsModelFit = statsModel.fit()
        ### PRINT RESULTS ###
        print(statsModelFit.summary())
        ### RETURN: resultsObject, y, X
        return statsModelFit, y, X
    
    ### MANN-WHITNEY-U ON TWO COLUMNS ###
    def stats_mwu(self,continuousColOne,continuousColTwo):
        testStat, pValue = sp.stats.mannwhitneyu(self.df[continuousColOne], self.df[continuousColTwo], use_continuity=True, alternative='two-sided')
        if self.verbose is True: print(" - Mann-Whitney-U: P="+str('%.2E' % Decimal(pValue))+" TS="+str(testStat)+" [ "+continuousColOne+" : "+continuousColTwo+" ] - ")
        return pValue, testStat 
    
    ### MANN-WHITNEY-U PAIRWISE CONTINUOUS VARIABLE BY CATEGORICAL COLUMN ###
    def stats_mwu_pairwise(self,continuousColumn,categoricalColumn):
        if self.verbose is True: print(" - Comparing groups of "+categoricalColumn+" by "+continuousColumn+" - ")
        resDF = DF(None, columnsIn=['group1','group2','p','t'])
        ### GET ALL PAIRWISE COMBINATIONS OF DISTANCE MATRICES ### 
        for catIDX, (cat1,cat2) in enumerate(itertools.combinations(self.df[categoricalColumn].unique(),2)):
            ### CALCULATE MANN-WHITNEY-U ###
            testStat, pValue = sp.stats.mannwhitneyu(self.df[self.df[categoricalColumn]==cat1][continuousColumn] , self.df[self.df[categoricalColumn]==cat2][continuousColumn], use_continuity=True, alternative='two-sided')
            if self.verbose is True: 
                if pValue > 0.00001: pPrint = str(pValue)
                else: pPrint = str('%.2E' % Decimal(pValue))
                print(" - Mann-Whitney-U: p="+ pPrint+" | t="+str(testStat)+" [ "+cat1+"(n="+str(len(self.df[self.df[categoricalColumn]==cat1][continuousColumn]))+") : "+cat2+"(n="+str(len(self.df[self.df[categoricalColumn]==cat2][continuousColumn]))+")] - ")
            resDF.df.loc[catIDX] = [cat1,cat2,pValue,testStat]
        return resDF
    ##########################################################################
    ############################ SET #####################################
    ##### .set_X SET VALUES IN DATAFRAME ##########################
    
    
    ### SET ###
    # INPUT: value - value to add to dataframe
    #              - can be None, int, str - fills whole df or rows or columns provided
    #              - can be list
    # INPUT COMBINATIONS:
    #   - if no column or row: fill whole dataframe with value
    #   - if column: fill column with value (can be int, str, None, or List[n=])
    def s(self, value, column=None, index=None, fillNone=None):
        
        # IF NO COLUMN OR INDEX PROVIDED... FILL WHOLE DF WITH VALUE
        if (column is None) and (index is None):self.df.loc[self.df.index][seld.df.columns] = value;return self
        
        # IF COLUMN PROVIDED...
        if (column is not None):
            # IF COLUMN IS LIST... 
            if isinstance(column, list):
                # FOR EACH COLUMN IN LIST... SET TO VALUE #
                for curCol in column: self.df[curCol] = value
            # IF COLUMN IS NOT LIST... TRY TO SET COLUMN TO VALUE #
            else: self.df[column] = value
        
        # IF INDEX PROVIDED...
        if (index is not None):
            # IF index IS LIST...
            if isinstance(index, list):
                # FOR EACH INDEX IN LIST... SET TO VALUE #
                for curIdx in index: self.df.loc[curIdx] = value
            # IF index IS NOT LIST... TRY TO SET COLUMN TO VALUE #
            else: self.df.loc[index] = value
        
        # SET BY INDEX AND COLUMN
        self.df.loc[index][column] = value
         
        ### IF NOT A DATAFRAME THEN CREATE WITH index AND columns FILLED BY value ###
        #self.df = pd.DataFrame(index=index, columns=column, dtype=None, copy=False);self.df.fillna(value)
        
        return self
    
    def set_(self, value, indexName, colName):
        self.df.set_value(indexName, colName, value); return self

    ### SET INDEX COLUMN OR COLUMNS ###
    def set_i(self, colNameOrList, drop=True, append=False):
        """
        USE: DF.set_index('name', dropIn=True, appendIn=False)
        INPUT:
           keys           : column name or list[column names]
           drop=True      : delete columns to become index
           append=False   : add to existing 
           inplace=False  : perform on target dataframe
        RETURN: None
        TYPE: function
        STRUCTURE: DF()
        """
        self.df.set_index(colNameOrList, drop=drop, append=append, inplace=True, verify_integrity=False); return self
    
    def set_r(self, newRow, index): self.df.loc[index] = newRow; return self
    def set_c(self, newCol, colName): self.df[colName] = newCol; return self
    
    ##########################################################################
    ############################# UPDATE #####################################
    ### UPDATE WRAPPER .i .index .c .columns ###
    def update(self):
        self.update_i() # Update Index .i .index
        self.update_c() # Update Columns .c .columns
        self.palette_rainbow(20)
        return self
    ### UPDATE INDEX .i ###
    def update_i(self): self.i = self.df.index; self.index = self.df.index; return self
    ### UPDATE COLUMNS .c ###
    def update_c(self): self.c = self.df.columns; self.columns = self.df.columns; return self
    
    
    ##########################################################################
    ############################## YIELD #####################################
    ##### .yield_X YIELD ITERABLES ##########################
    
    ### YIELD BY INDICES (rows) ###
    def yield_i(self): 
        for iCur in self.i: yield self.df.loc[[iCur]]
    ### YIELD BY COLUMNS (cols) ###
    def yield_c(self): 
        for cCur in self.c: yield self.df[[cCur]]
    
    
    ##########################################################################
    ############################## PLAYPEN ###################################
    
    #### ADD COLUMN TO DATAFRAME ####
    #def add_c(self, newColumn, name=None, overwrite=True):
    #    newColumn = pd.Series(data=newColumn, index=self.df.index, dtype=None, name=name, copy=False)
    #    self.df = pd.concat([self.df,newColumn], axis=0, join='outer', join_axes=useIndex, ignore_index=overwrite,
    #                            keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    #### ADD ROW TO DATAFRAME ###
    #def add_r(self, newRow, name=None, overwrite=True):    
    #    newRow = pd.Series(data=newRow, index=self.df.columns, dtype=None, name=name, copy=False)
    #    self.df = pd.concat([self.df,newRow], axis=0, join='outer', join_axes=None, ignore_index=overwrite,
    #                            keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    
    #### MERGE DATAFRAME CLASS OBJECT INTO CURRENT OBJECT ###
    #def merge(self, inDF, useIndex=None, unionIn=False, replaceColnames=False):
    #    if unionIn is true: joinIn = 'inner'
    #    else: joinIn = 'outer'
    #    return DF(pd.concat([self.df,inDF], axis=1, join=joinIn, join_axes=useIndex, ignore_index=overwrite,
     
    ##########################################################################
    ############################ TODO ########################################
    
    ### MISSINGNESS TOOLS - BEAUTIFUL FIGURES 
    #import missingno as msno
    #msno.matrix(collisions.sample(250))
        
### EXAMPLE USAGE ###
#X = DF("test_data/ag_analysis/1_1_qc_1000_map.txt")    
#X = DF(None, columnsIn=["Ace","Duce","Cinco"], indexIn=np.arange(6))
### PERFORM MANN-WHITNEY-U ON PAIRSWISE COMBINATIONS OF AGE BY RACE ###
# X.stats_mwu_pairwise('age_years','race')
#X.df
### PLOT A LINEAR MODEL ###
#f = X.plot_violin(categoricalColX='race',continuousColY='age_years',categoricalColor='economic_region')
#g = X.plot_lm(continuousColX='age_years',continuousColY='bmi',categoricalColor='economic_region')#,categoricalColumn='race',categoricalRow='sex')
#X.stats_regression("bmi ~ age_years + race + sex")[0]
#X.set_c(colName='winning',newCol=np.arange(len(X.i))).d()
#X.set_r(index='winning',newCol=).d()
#X.set_r(index=6, newRow = ['Nan','Nan','Nan'])
#X.update()

### LOAD MAPPING FILE & SET SAMPLID TO INDEX ###
#mapX = DF(mapPath,sepIn='\t',verboseIn=True)
#mapX.set_index("#SampleID")

### EXAMPLE USAGE ###
# SET DISPLAY TO FLOAT #
#mapX.display_float()

# PERFORM PAIRWISE MANN-WHITNEY-U #
#mwuRes = mapX.stats_mwu_pairwise(categoricalColumn='race',continuousColumn='bmi')
#mwuRes.out_tsv('t98.txt'); mwuRes.display()

# PERFORM REGRESSION #
#mapX.stats_regression("age_years ~ race")

# PLOT VIOLIN #
#mapX.plot_violin(categoricalColX='sex',continuousColY='bmi',categoricalColor='race')

# PLOT LINEAR MODEL #
#xLM = mapX.plot_lm(continuousColX='age_years',continuousColY='bmi',categoricalColor='economic_region')
#xLM.savefig('t99.pdf')

# x.s(0, index=[2,3,6],column=['Duce','Ave']).df

<h4 style="text-align:center; color:blue;"> - Load Tree Tools - </h4>

In [5]:
####### TREE TOOLS (TreeNode - skbio) #####################################################

### LOAD NEWICK TREE (i.e. BACTERIAL PHYLOGENY) INTO TREENODE OBJECT ###
def tree_load(trePathIn): 
    print(" - Loading Bacterial Phylogeny - " + trePathIn)
    ### LOAD TREE USING SKBIO TREENODE OBJECT ###
    treeInIn = TreeNode.read(trePathIn)
    ### TRAVERSE TREE AND SET NONE TO 0.0 (AVOID ERRORS DOWNSTREAM) ###
    for idx, e in enumerate(treeInIn.traverse()): 
        if e.length == None: e.length = 0.0
    return treeInIn

### TRIM TREE TO OTUS IN BIOM TABLE ###
def tree_filter_biom(btIn, treeFIn):
    """
    tree_filter_biom is adapted from the GNEISS PACKAGE
    Morton JT... Knight R. 2017. Balance trees reveal microbial niche differentiation. 
    mSystems 2:e00162-16. https://doi.org/10.1128/mSystems.00162-16.
    DOWNLOADED FROM: https://github.com/biocore/gneiss/
    """
    print(" - Filtering Bacterial Phylogeny to OTUs in BIOM Table - ")
    treeFIn2 = treeFIn.shear(names=btIn.ids(axis='observation'))
    treeFIn2.bifurcate()
    treeFIn2.prune()
    return treeFIn2

### WRITE TREE TO NEWICK FILE ###
def tree_write(treeInOut, treePathOut):
    print(" - Writing Tree to Newick File " + treePathOut + " - ")
    treeInOut.write(treePathOut, format='newick')
    return

<h4 style="text-align:center; color:blue;"> - Load Distance Matrix Tools - </h4>

In [6]:
####### DISTANCE MATRIX (DistanceMatrix - skbio) ##########################################

### WRITE DISTANCE MATRIX ###
def dm_write(dmIn, savePath): dmIn.write(savePath, format='lsmat')

##### SPLIT DISTANCE MATRIX TO ALL COMBINATIONS OF GROUPS IN A METADATA CATEGORY #####
# pairwise, triwise...
# i.e. table with groups: g1, g2, g3
# [g1&g2, g2&g3, g1&g3]
# TO USE:
# dmS = dm_metadata_combinations(consensusDM, mapDf, 'race')
# for i in dmS:
#    print(i[0]) # NAME OF GROUPS
#    i[1]  # DISTANCE MATRIX
def dm_metadata_combinations(dmIn, mapDfIn, mapCatIn):
    ### FOR EACH LEVEL OF COMBINATIONS ###
    for combNumber in np.arange(2,len(mapDfIn[mapCatIn].unique())+1):
        print(" - All Table Combinations of "+str(combNumber)+" groups - ")
        
        ### FOR EACH SET OF COMBINATIONS AT THAT LEVEL ###
        for combPairs in itertools.combinations(mapDfIn[mapCatIn].unique(),combNumber):
            print("   - Group: "+str(combPairs))
            
            ### GET SAMPLES IN GROUPS ###
            samplesInGroups = []
            for combGroup in combPairs:samplesInGroups.extend(mapDfIn[mapDfIn[mapCatIn]==combGroup].index)
            
            ### FILTER DISTANCE MATRIX ###
            yield combPairs, dmIn.filter(samplesInGroups, strict=False)


<h4 style="text-align:center; color:blue;"> - Load Plotting Tools - </h4>

In [7]:
####### PLOTTING (Various) ################################################################

#########################################################################################
##### ADDITIONAL FUNCTIONS #####

### OPEN INTERFACE TO SELECT CONTINOUS SEQUENTIAL COLORMAP ###
#continuousColorMap = sns.choose_colorbrewer_palette('sequential', as_cmap=True)

### OPEN INTERFACE TO SELECT CONTINOUS DIVERGING COLORMAP ###
#continuousColorMap = sns.choose_colorbrewer_palette('diverging', as_cmap=True)

### OPEN INTERFACE TO SELECT CATEGORICAL COLOR PALETTE (NOT COLORMAP) ###
#categoricalColorPalette = sns.choose_colorbrewer_palette('qualitative', as_cmap=False)

# fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True)
# ax1.plot(x)

PAP = """
def plot_area_pie(dfIn, radiiContinuousColumn=None, widthContinuousColumn=None,):
    
    ### GET EVENLY DISTRIBUTED LINESPACE IN DIMENSION AND RANGE OF RADII LINESPACE ###
    thetaX = np.linspace(np.min(dfIn[radiiContinuousColumn]),np.max(dfIn[radiiContinuousColumn]), len(dfIn[radiiContinuousColumn]), endpoint=False)
    print(thetaX)
    ### INITIALIZE PLOTSPACE ###
    ax = plt.subplot(111, projection='polar')
    
    ### GENERATE PLOT ###
    bars = ax.bar(thetaX, dfIn[radiiContinuousColumn], width=(dfIn[widthContinuousColumn]/100), bottom=0.0)

    ### SET COLOR AND ALPHA ###
    for r, bar in zip(dfIn[radiiContinuousColumn], bars):
        bar.set_facecolor(plt.cm.viridis(8 / 10))
        bar.set_alpha(0.5)
        
    ### SAVE AND SHOW ###
    plt.show()
    return
plot_area_pie(mapDf[mapDf['race']=='Hispanic'], radiiContinuousColumn='age_years',widthContinuousColumn='bmi')
"""
# HERE

<h3 style="text-align:center; color:blue;"> - Load BIOM Table Tools - </h3>

In [8]:
####### BIOM TABLE (Table - Biom) - I/O ###################################################

### READ BIOM TABLE FROM JSON OR TSV OBJECT ###
def table_in(filePath): return load_table(filePath)

### WRITE OTU TABLE TO TSV ###
def table_tsv(bt, fileName, printOut=False):
    if printOut == True: print(" - Writing BIOM Table TSV - " + fileName)
    f = open(fileName,'w')
    f.write(bt.to_tsv())
    f.close()

### WRITE OTU TABLE AS JSON ###
def table_json(bt, fileName, printOut=False):
    if printOut == True: print(" - Writing BIOM Table JSON - " + fileName)
    f = open(fileName,'w')
    f.write(bt.to_json("biom", direct_io=None))
    f.close()


In [9]:
####### BIOM TABLE (Table - Biom) - INFO ##################################################

### RETURN LIST OF OBSERVATIONS ###
def table_observations(bt, writePath=None): 
    otusInF = bt.ids(axis='observation')
    if writePath != None: 
        outListFile = open(writePath, 'w') 
        for itemCycle in otusInF: outListFile.write("%s\n" % itemCycle)
        outListFile.close() 
    return otusInF

### Return List of Observation Total Counts ###
def table_observations_counts(bt, writePath=None): 
    otusCountsInF = bt.sum('observation')
    if writePath != None: 
        outListFile = open(writePath, 'w')
        for itemCycle in otusCountsInF: outListFile.write("%s\n" % itemCycle)
        outListFile.close() 
    return otusCountsInF

### Return List of Samples ###
def table_samples(bt, writePath=None): 
    samplesInF = bt.ids(axis='sample')
    if writePath != None: 
        outListFile = open(writePath, 'w')
        for itemCycle in samplesInF: outListFile.write("%s\n" % itemCycle)
        outListFile.close() 
    return samplesInF

### Return List of Counts for Each Sample ###
def table_samples_counts(bt, writePath=None): 
    samplesCountsInF = bt.sum('sample')
    if writePath != None: 
        outListFile = open(writePath, 'w')
        for itemCycle in samplesCountsInF: outListFile.write("%s\n" % itemCycle)
        outListFile.close() 
    return samplesCountsInF

### GET GENERAL INFO ABOUT TABLE ###
def table_info(bt, writePath=None, printOut = True):
    otusIn = table_observations(bt, writePath=None)
    otusCountsIn = table_observations_counts(bt, writePath=None)
    samplesIn = table_samples(bt, writePath=None)
    samplesCountsIn = table_samples_counts(bt, writePath=None)
    if printOut == True:
        print(" - Table Info - ")
        print('   Total Observations: '+str(len(otusIn)))
        print('   Total Samples: '+str(len(samplesIn)))
        print('   Total Counts: '+str(bt.sum()))
        print('   Non-Zero Entries: '+str(bt.nnz))
        print('   Table Density: '+str(bt.get_table_density()))
    if writePath != None: 
        fOut = open(writePath,'w')
        fOut.write(" - Table Info - \n")
        fOut.write("   Total Observations: "+str(len(otusIn))+"\n")
        fOut.write("   Total Samples: "+str(len(samplesIn))+"\n")
        fOut.write("   Total Counts: "+str(bt.sum())+"\n")
        fOut.write("   Non-Zero Entries: "+str(bt.nnz)+"\n")
        fOut.write("   Table Density: "+str(bt.get_table_density()))
        fOut.close()
    return otusIn,otusCountsIn,samplesIn,samplesCountsIn

### GET METADATA CATEGORIES ###
def table_metadata(bt, mappingFileDataframe):
    print(" - Getting Metadata Categories - ")
    # GET METADATA CATEGORIES FROM MAPPING FILE #
    metaC = []
    for i in mappingFileDataframe.columns: metaC.append(i)
    # CREATE METADATA DICTIONARY FORMAT ALLOWING TO ADD TO BIOM TABLE STRUCTURE #
    print(" - Constructing Dictionary of Metadata by Samples - ")
    iterSamples = bt.iter(axis='sample'); metaD={}
    # Loop through samples #
    
    for values, idCur, metadata in iterSamples: 
        metaD[idCur] = {}
        for idx, i in enumerate(mappingFileDataframe.loc[idCur]): 
            metaD[idCur][metaC[idx]] = i            
    return metaC, metaD

### PIPELINE TO WRITE TABLE TO TSV AND OUTPUT INFO & LISTS ###
# EACH FILE WILL BE SAVED WITH tableID_table_(file description).txt IN dirPath FOLDER
def table_write(bt, outDirPath, tableID, toTSV=True, toJSON=False):
    ##### WRITE INPUT BIOM TABLE TO TSV #####
    if toTSV == True: table_tsv(bt, outDirPath+tableID+"_table.txt", printOut = True)
    ##### WRITE INPUT BIOM TABLE TO JSON ##### (Sometimes errors with metadata)
    if toJSON == True: table_json(bt, outDirPath+tableID+"_table.biom", printOut=True)
    ##### PRINT AND WRITE TABLE INFO TO FILE #####
    table_info(bt, writePath=outDirPath+tableID+"_table_summary.txt", printOut = False)
    ##### WRITE SAMPLE LIST #####
    table_observations(bt, writePath=outDirPath+tableID+"_table_otus.txt")
    ##### WRITE SAMPLE COUNTS #####
    table_observations_counts(bt, writePath=outDirPath+tableID+"_table_otus_counts.txt")
    ##### WRITE OTUS LIST #####
    table_samples(bt, writePath=outDirPath+tableID+"_table_samples.txt")
    ##### WRITE SAMPLES COUNTS #####
    table_samples_counts(bt, writePath=outDirPath+tableID+"_table_samples_counts.txt")
    return


In [10]:
####### BIOM TABLE (Table - Biom) - FILTERING #############################################

##### PIPELINE TO PERFORM BASIC QUALITY CONTROL - WILL RETURN FILTERED BIOM TABLE & MAP #####
### DEFAULT REMOVES EMPTY OTUS AND SAMPLES ###
def table_qc(bt, minimumOTUCountIn=1, minimumSampleCountIn=1, minimumOTUUbiquityIn=0.0):
    ##### FILTER OTUS BY MINIMUM COUNT #####
    bt = table_filter_otu_mincount(bt, minimumOTUCountIn)
    ##### FILTER OTUS BY MINIMUM SAMPLES #####
    if minimumOTUUbiquityIn > 0.0: bt = filter_otu_minubiquity(bt, minimumOTUUbiquityIn)
    ##### FILTER SAMPLES BY MINIMUM COUNT #####
    bt = table_filter_sample_mincount(bt, minimumSampleCountIn)
    ##### FILTER OTUS BY MINIMUM COUNT (AGAIN TO GUARENTEE ALL HAVE 10+ COUNTS AFTER SAMPLE FILTER) #####
    bt = table_filter_otu_mincount(bt, minimumOTUCountIn)
    ##### FILTER SAMPLES BY MINIMUM COUNT (AGAIN TO GUARENTEE SAMPLES IN TABLE ARE SAME AS RAREFIED) #####
    bt = table_filter_sample_mincount(bt, minimumSampleCountIn)
    return bt

### FILTER OTU MINCOUNT ### - remove otus with < mincount
def table_filter_otu_mincount(bt, mincount):
    filter_func = lambda values, id, md: sum(values) >= mincount
    return bt.filter(filter_func, axis='observation', inplace=False)

### FILTER OTU MAXCOUNT ### - remove otus with > maxcount
def table_filter_otu_maxcount(bt, maxcount):
    filter_func = lambda values, id, md: sum(values) <= maxcount
    return bt.filter(filter_func, axis='observation', inplace=False)

### FILTER SAMPLE MINCOUNT ### - remove otus with < mincount
def table_filter_sample_mincount(bt, mincount):
    filter_func = lambda values, id, md: sum(values) >= mincount
    return bt.filter(filter_func, axis='sample', inplace=False)

### FILTER SAMPLE MAXCOUNT ### - remove otus with > maxcount
def table_filter_sample_maxcount(bt, maxcount):
    filter_func = lambda values, id, md: sum(values) <= maxcount
    return bt.filter(filter_func, axis='sample', inplace=False)

### FILTER OTU MINIMUM UBIQUITY BY PERCENT SAMPLES ### - remove otus in < minubiq fraction of samples
def table_filter_otu_minubiquity(bt, minubiq):
    filter_func = lambda val, id_, md: sum(val>0)/len(val) >= minubiq
    return bt.filter(filter_func, axis='observation', inplace=False)

### FILTER OTU MAXIMUM UBIQUITY BY PERCENT SAMPLES ### - remove otus in > maxubiq fraction of samples
def table_filter_otu_maxubiquity(bt, maxubiq):
    filter_func = lambda val, id_, md: sum(val>0)/len(val) <= maxubiq
    return bt.filter(filter_func, axis='observation', inplace=False)
    
### FILTER OTU LISTKEEP ### - remove all otus from table not im listkeep
def table_filter_otu_listkeep(bt, list_to_keep):
    filter_func = lambda values, id, md: id in list_to_keep
    return bt.filter(filter_func, axis='observation', inplace=False)

### FILTER OTU LISTREMOVE ### - give a list of otu's to remove from table
def table_filter_otu_listremove(bt, list_to_remove):
    filter_func = lambda values, id, md: id not in list_to_remove
    return bt.filter(filter_func, axis='observation', inplace=False)

### FILTER SAMPLE LISTKEEP ### - remove all otus from table not in listkeep
def table_filter_sample_listkeep(bt, list_to_keep): 
    filter_func = lambda values, id, md: id in list_to_keep 
    return bt.filter(filter_func, axis='sample', inplace=False)

### FILTER SAMPLE LISTREMOVE ### - give a list of otu's to remove from table
def table_filter_sample_listremove(bt, list_to_remove):
    filter_func = lambda values, id, md: id not in list_to_remove
    return bt.filter(filter_func, axis='sample', inplace=False)

### FILTER TO SAMPLES IN METADATA CATEGORY:GROUP ### - keep only samples in the specified group
def table_filter_metadata_contain(bt, metadata_category, metadata_group):
    filter_f = lambda values, id_, md: md[metadata_category] == metadata_group
    return bt.filter(filter_f, axis='sample', inplace=False)

### FILTER TO SAMPLES NOT IN METADATA CATEGORY:GROUP ### - keep only samples not in the specified group 
def table_filter_metadata_exclude(bt, metadata_category, metadata_group):
    filter_f = lambda values, id_, md: md[metadata_category] != metadata_group
    return bt.filter(filter_f, axis='sample', inplace=False)


In [11]:
####### BIOM TABLE (Table - Biom) - RAREFACTION ###########################################

# Take in BIOM Table and Subsample each Sample to samplingDepth
# Returns Rarefied BIOM Table Object
def table_rarefaction(bt, samplingDepth = 1000, warnSkippedSamples=False):
    dataCounts = []; samplesDataCounts = []
    # Iterate over All of the Samples #
    iterSamples = bt.iter(axis='sample')
    for values, id, metadata in iterSamples:
        # Subsample to the Specified Depth #
        if sum(values) >= samplingDepth:
            # Store Subsampled Counts for Sample #
            dataCounts.append(sk.stats.subsample_counts(values.astype(int), samplingDepth, replace=False))
            # Store Sample ID #
            samplesDataCounts.append(id)
        # If Sample has < Counts than samplingDepth: Print Warning #
        elif warnSkippedSamples==True: print("   Warning Skipped Sample : ", id)
    # Return BIOM Table Object #
    return Table(np.matrix(dataCounts).T, bt.ids(axis='observation'), samplesDataCounts)
    

### FUNCTION TO PERFORM MULTIPLE RAREFACTIONS ON AN OTU TABLE AT AN EVEN DEPTH ###
def table_rarefactions_even_depth(bt, depthRare, numRare):
    ### ARRAY TO STORE RAREFIED TABLES ###
    rareArray = []
    ### RAREFY TABLES ###
    for numRareIncrement in np.arange(numRare): rareArray.append(rarefaction(bt, samplingDepth = depthRare, warnSkippedSamples=False))
    return rareArray

### FUNCTION WRAPPER TO PERFORM MULTIPLE RAREFACTIONS ON AN OTU TABLE AT MULTIPLE DEPTHS ###
def table_rarefactions_multiple_depths(bt, minDepthRare, maxDepthRare, rareStep, numRare):
    ### DICTIONARY TO HOLD ARRAYS OF RAREFIED TABLES AT EACH DEPTH ###
    rareDictIn = {}
    ### LOOP THROUGH RAREFACTIONS AT EACH DEPTH...
    for rareCurDepth in np.arange(minDepthRare, maxDepthRare, rareStep):
        ### GET LIST OF RAREFIED TABLES AT THAT DEPTH ###
        rareDictIn[rareCurDepth] = rarefactions_even_depth(bt, rareCurDepth, numRare)
    return rareDictIn


In [12]:
####### BIOM TABLE (Table - Biom) - TRANSFORMATION ########################################

### RETURN RELATIVE ABUNDANCE TABLE ###
def table_relative(bt):
    tablerel = bt.norm(axis='sample', inplace=False)
    return tablerel

### GET BIOMTABLE COUNTS AS MATRIX WITH SAMPLES AS COLUMNS (TRANSFORM ROW<->COLS AND CONVERT TO INT) ###
def table_array(bt): return bt.matrix_data.todense().T

### BIOM COUNTS AS PANDAS DATAFRAME ###
def table_dataframe(bt): return pd.DataFrame(bt.matrix_data.todense().T.astype('float'), index=bt.ids(axis='sample'), columns=bt.ids(axis='observation'))

### GET OTU TAXONOMY AS PANDAS DATAFRAME ###
def table_dataframe_taxa(bt):
    metaDictTaxonomy = {}; iterOTUs = bt.iter(axis='observation')
    ### ITERATE OVER OTUS AND CREATE DICTIONARY OF TAXONOMY ###
    for idx, (values, id, metadata) in enumerate(iterOTUs): metaDictTaxonomy[id] = metadata['taxonomy']
    ### CREATE AND RETURN AS PANDAS DATAFRAME ###
    dfTaxPD = pd.DataFrame.from_dict(metaDictTaxonomy, orient='index')
    dfTaxPD.columns = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
    return dfTaxPD

### PARTITION SAMPLES BY METADATA CATEGORY ### -> Dictionary of Tables
def table_partition_metadata(bt, metadata_category, print_summary=False):
    print(" - Partitioning Table by " + metadata_category + " - ")
    part_f = lambda id_, md: md[metadata_category]
    partTabs = bt.partition(part_f, axis='sample')
    partTables = {}
    for partition, partTab in partTabs: 
        partTables[partition] = partTab
        if print_summary == True: print(partition)
        if print_summary == True: table_info(partTab)
        if print_summary == True: print(str(partTab.sum('sample')))
    return partTables

### COLLAPSE SAMPLES BY METADATA CATEGORY ###
def table_collapse_metadata(bt, metaDataCatIn):
    ### DICTIONARY OF COUNTS FOR EACH METADATA GROUPING ###
    metaCountsCollapsed = {}; iterSamples = bt.iter(axis='sample')
    ### ITERATE OVER SAMPLES ###
    for idx, (values, id, metadata) in enumerate(iterSamples):
        ### CHECK IF METADATA GROUP IN DICTIONARY AND INITIALIZE COUNTS IF NOT ###
        if metadata[metaDataCatIn] not in metaCountsCollapsed.keys(): metaCountsCollapsed[metadata[metaDataCatIn]] = values
        ### ELSE ADD THE COUNTS TO DICTIONARY ###
        else: metaCountsCollapsed[metadata[metaDataCatIn]] += values
    ### COUNTS AS ARRAY OF ARRAYS AND METADATA IDS AS LIST AND RETURN TO BIOM TABLE FORMAT ###
    metaCountIds = []; metaCountMatrix = []
    for metaLoopId in metaCountsCollapsed.keys(): metaCountIds.append(metaLoopId); metaCountMatrix.append(metaCountsCollapsed[metaLoopId])
    return Table(np.array(metaCountMatrix).T, bt.ids(axis='observation'), metaCountIds, observation_metadata=None, sample_metadata=None, table_id=None, type=None, create_date=None, generated_by=None, observation_group_metadata=None, sample_group_metadata=None) 

### COLLAPSE OTUs AT TAXONOMIC LEVEL ###
# tax_level: 0 = Kingdom | 1 = Phylum | 2 = Class | 3 = Order | 4 = Family | 5 = Genus | 6 = Species
def table_collapse_taxonomy(bt, tax_level):
    collapse_f = lambda id_, md: '; '.join(md['taxonomy'][:tax_level + 1])
    tabletaxacollapse = bt.collapse(collapse_f, axis='observation',norm=False)
    return tabletaxacollapse

### CONVERT TO PRESENCE / ABSENCE TABLE ###
# Table contains 1 if count >0 or 0
def table_presence_absence(bt): return bt.pa()

In [13]:
####### BIOM TABLE (Table - Biom) - ALPHA DIVERSITY #######################################
##### COMPUTES ALPHA DIVERSITY METRICS FOR BIOM TABLE #####
# !!! DO NOT RUN ON RELATIVE ABUNDANCE - DEPENDENT ON INT CONVERSION !!! #
def alpha_diversity(bt, alphaMetricsIn=['chao1','observed_otus','shannon','simpson'], vocalAlphaIn=True):
    alphaResultsDict = {}
    ### FOR EACH ALPHA METRIC...
    for alphaMetricCycle in alphaMetricsIn:
        if vocalAlphaIn == True: print(" - Calculating Alpha Diversity Metric "+alphaMetricCycle+" - ")
        ### CALCULATE ALPHA DIVERSITY AND STORE IN DICT ###
        alphaResultsDict[alphaMetricCycle] = sk.diversity.alpha_diversity(alphaMetricCycle, table_array(bt).astype(int), ids=bt.ids(axis='sample'), validate=True)
    ### AND RETURN AS PANDAS DICTIONARY ###
    return pd.DataFrame.from_dict(alphaResultsDict)

### CALCULATES ALPHA DIVERSITY FOR RAREFIED TABLES ###
def alpha_pipeline(rareTablesIn):
    alphaDictSum = None
    ### FOR EACH RAREFIED TABLE...
    for idx, rareTableCycle in enumerate(rareTablesIn):
        ### CALCULATE ALPHA DIVERSITY AND SUM TOGETHER ACROSS TABLES ###
        if idx == 0: alphaDictSum = alpha_diversity(rareTableCycle, vocalAlphaIn=True); print(" - Alpha Pipeling Continuing Silently for All Tables - ")
        else: alphaDictSum += alpha_diversity(rareTableCycle, vocalAlphaIn=False)
    ### DIVIDE BY NUMBER OF TABLES AND RETURN ###
    return alphaDictSum / (idx+1) 

##### ALPHA DIVERSITY ANALYSES #####
# GENERATES PLOTS OF ALPHA DIVERSITY RESULTS FOR SAMPLES #
# Requires a dataframe of alpha diversity for each sample and mapping file
# Provide: mapCat1 as a categorical column in mapDfIn
# Provide: mapCat2 to examine how alpha diversity structures by both factors
# Provide: savePath where alpha diversity results will be saved
def alpha_pipeline_analysis(alphaDfIn, mapDfIn, mapCat1, mapCat2=None, metricsIn = ['shannon','observed_otus','chao1','simpson'],
                         savePath=None, plotDisplay = True, plotIndividualMetrics = True, plotPairplotAllMetrics = False):
    
    ### SAVE TO PDF IF PATH IS PASSED ###
    if (savePath != None):
        # SAVE CORRELATION MATRIX OF ALPHA METRICS #
        alphaDfIn.corr(method="spearman").to_csv(savePath+'_alpha_correlation.txt',sep='\t')
        # IF SECOND METRIC PASSED ADD TO NAME WITH FIRST METRIC #
        if mapCat2 != None: pdf = PdfPages(savePath+"_alpha_"+mapCat1+"_"+mapCat2+".pdf")
        # ELSE SAVE WITH ONLY FIRST METRIC #
        else: pdf = PdfPages(savePath+"_alpha_"+mapCat1+".pdf")
        
    ### TEMPORARLY DISABLE WARNINGS ###
    import warnings; warnings.filterwarnings('ignore')
    #pd.set_option('display.width', 1000)
    
    ### ADD FIRST MAP CATEGORY TO ALPHA DF ###
    alphaDfIn[mapCat1] = mapDfIn[mapCat1]
    ### ADD SECOND MAP CATEGORY TO ALPHA DF ###
    if mapCat2 != None: alphaDfIn[mapCat2] = mapDfIn[mapCat2]
        
    ### GET MEAN DIVERSITIES FOR CATEGORY ONE ###
    if (savePath != None) or (plotDisplay == True): meanCat1DF = alphaDfIn.groupby(mapCat1).mean()
    if savePath != None: meanCat1DF.to_csv(path_or_buf=savePath+"_alpha_mean_"+mapCat1+".txt", sep='\t')
    if plotDisplay == True: print(" - Mean Alpha Diversity by "+mapCat1+" - "); display(meanCat1DF)
    ### GET MEAN DIVERSITIES FOR CATEGORY TWO ###
    if mapCat2 != None:
        if (savePath != None) or (plotDisplay == True): meanCat2DF = alphaDfIn.groupby(mapCat2).mean()
        if savePath != None: meanCat2DF.to_csv(path_or_buf=savePath+"_alpha_mean_"+mapCat2+".txt", sep='\t')
        if plotDisplay == True: print(" - Mean Alpha Diversity by "+mapCat2+" - "); display(meanCat2DF)
        ### AND BOTH CATEGORIES TOGETHER ###
        if (savePath != None) or (plotDisplay == True): meanCatBothDF = alphaDfIn.groupby([mapCat1,mapCat2]).mean()
        if savePath != None: meanCatBothDF.to_csv(path_or_buf=savePath+"_alpha_mean_"+mapCat1+"_"+mapCat2+".txt", sep='\t')
        if plotDisplay == True: print(" - Mean Alpha Diversity by "+mapCat1+" and "+mapCat2+" - "); display(meanCatBothDF)

    ##### INDIVIDUAL PLOTS #####
    # DISTRIBUTION PLOTS FOR EACH INDIVIDUAL METRIC #
    if plotIndividualMetrics == True:
        fig = plt.subplots(4,1,figsize=[10,8])
        plt.title("Metric Distributions")
        ### FOR EACH METRIC GENERATE PLOT ###
        for idx, i in enumerate(metricsIn):
            plt.subplot(4,1,idx+1)
            sns.distplot(alphaDfIn[i], rug=True)
            ymin, ymax = plt.gca().get_ylim(); plt.vlines(np.mean(alphaDfIn[i]),ymin,ymax,color='r')
            curAx = plt.gca(); curAx.axes.get_yaxis().set_visible(False)
            plt.xlabel(i)
        plt.tight_layout(); 
        if savePath != None: pdf.savefig()
        if plotDisplay == True: plt.show(); 
        plt.close()
    
    ##### PAIRPLOT ALL METRIC COLOR BY CATEGORIES #####
    if plotPairplotAllMetrics == True:
        gbX = sns.pairplot(data=alphaDfIn, hue=mapCat1, vars=metricsIn); 
        if savePath != None: pdf.savefig()
        if plotDisplay == True: plt.show(gbX); 
        plt.close()
        ### SECOND CATEGORY ###
        if mapCat2 != None: 
            gbX = sns.pairplot(data=alphaDfIn, hue=mapCat2, vars=metricsIn); 
            if savePath != None: pdf.savefig()
            if plotDisplay == True: plt.show(gbX); 
            plt.close()
            
    ##### BOXPLOTS #####
    for boxplotMetric in metricsIn:
        ### PLOT ALL SAMPLES TOGETHER ###
        plt.figure(figsize=[10,5]); 
        sns.boxplot(y=boxplotMetric, data=alphaDfIn, saturation=0.5, meanline=True, showmeans=True);
        plt.title("All Individuals"); plt.tight_layout();
        if savePath != None: pdf.savefig()
        if plotDisplay == True: plt.show(); 
        plt.close()
            
        ### BOXPLOT PLOT SPLIT BY MAPCAT1 ###
        if mapCat1 != None: 
            plt.figure(figsize=[10,5]); sns.boxplot(x=mapCat1, y=boxplotMetric, data=alphaDfIn, saturation=0.5, meanline=True, showmeans=True); 
            plt.title("Individuals by "+mapCat1); plt.tight_layout();
            # CALCULATE STATS #
            statStringOut = ("ALPHA STATISTIC COMPARISON FOR METRIC "+boxplotMetric+" by "+mapCat1+"\r")
            for manuResult in pandas_stats_mannwhitneyu(alphaDfIn, boxplotMetric, mapCat1): statStringOut += (list_to_string(manuResult)+"\r")
            list_write([statStringOut], savePath+"_alpha_statistics_"+boxplotMetric+"_"+mapCat1+".txt")
            # SAVE AND CLOSE #
            if savePath != None: pdf.attach_note(statStringOut, positionRect=[-100, -100, 0, 0]); pdf.savefig()
            if plotDisplay == True: print(statStringOut); plt.show(); 
            plt.close()
    
        if mapCat2 != None:
            ### BOXPLOT SPLIT BY MAPCAT2 ###
            plt.figure(figsize=[10,5]); sns.boxplot(x=mapCat2, y=boxplotMetric, data=alphaDfIn, saturation=0.5, meanline=True, showmeans=True);
            plt.title("Individuals by "+mapCat2); plt.tight_layout();
            # CALCULATE STATS #
            statStringOut = ("ALPHA STATISTIC COMPARISON FOR METRIC "+boxplotMetric+" by "+mapCat2+"\r")
            for manuResult in pandas_stats_mannwhitneyu(alphaDfIn, boxplotMetric, mapCat2): statStringOut += (list_to_string(manuResult)+"\r")
            list_write([statStringOut], savePath+"_alpha_statistics_"+boxplotMetric+"_"+mapCat2+".txt")
            # SAVE AND CLOSE #
            if savePath != None: pdf.attach_note(statStringOut, positionRect=[-100, -100, 0, 0]); pdf.savefig()
            if plotDisplay == True: print(statStringOut); plt.show(); 
            plt.close()
        
            ####### BOXPLOT SPLIT BY MAPCAT1 AND MAPCAT2 #######
            plt.figure(figsize=[10,5]); sns.boxplot(x=mapCat1, y=boxplotMetric, hue=mapCat2, data=alphaDfIn, saturation=0.5, meanline=True, showmeans=True);
            plt.title("Individuals by "+mapCat1+" and "+mapCat2);plt.tight_layout();
            # CALCULATE STATS #
            statStringOut = ("ALPHA STATISTIC COMPARISON FOR METRIC "+boxplotMetric+" by "+mapCat1+" and "+mapCat2+"\r")
            # FOR EACH GROUP IN FIRST CATEGORY... CALCULATE SECOND CATEGORY STATS ###
            for cat1Cycling in alphaDfIn[mapCat1].unique():
                statStringOut += ("\rWithin: "+cat1Cycling+"\r")
                for manuResult in pandas_stats_mannwhitneyu(alphaDfIn[alphaDfIn[mapCat1]==cat1Cycling], boxplotMetric, mapCat2): statStringOut += (list_to_string(manuResult)+"\r")
            list_write([statStringOut], savePath+"_alpha_statistics_"+boxplotMetric+"_"+mapCat1+"_"+mapCat2+".txt")
            # SAVE AND CLOSE #
            if savePath != None: pdf.attach_note(statStringOut, positionRect=[-100, -100, 0, 0]); pdf.savefig()
            if plotDisplay == True: print(statStringOut); plt.show(); 
            plt.close()
        
            ####### BOXPLOT SPLIT BY MAPCAT2 AND MAPCAT1 #######
            plt.figure(figsize=[10,5]); sns.boxplot(x=mapCat2, y=boxplotMetric, hue=mapCat1, data=alphaDfIn, saturation=0.5, meanline=True, showmeans=True);
            plt.title("Individuals by "+mapCat2+" and "+mapCat1);plt.tight_layout();
            # CALCULATE STATS #
            statStringOut = ("ALPHA STATISTIC COMPARISON FOR METRIC "+boxplotMetric+" by "+mapCat2+" and "+mapCat1+"\r")
            # FOR EACH GROUP IN FIRST CATEGORY... CALCULATE SECOND CATEGORY STATS ###
            for cat1Cycling in alphaDfIn[mapCat2].unique():
                statStringOut += ("\rWithin: "+cat1Cycling+"")
                for manuResult in pandas_stats_mannwhitneyu(alphaDfIn[alphaDfIn[mapCat2]==cat1Cycling], boxplotMetric, mapCat1): statStringOut += (list_to_string(manuResult)+"\r")
            list_write([statStringOut], savePath+"_alpha_statistics_"+boxplotMetric+"_"+mapCat2+"_"+mapCat1+".txt")
            # SAVE AND CLOSE #
            if savePath != None: pdf.attach_note(statStringOut, positionRect=[-100, -100, 0, 0]); pdf.savefig()
            if plotDisplay == True: print(statStringOut); plt.show(); 
            plt.close()
            
    if savePath != None: pdf.close()
    
    ### RESET WARNINGS ###
    warnings.filterwarnings('default')
    return


In [14]:
####### BIOM TABLE (Table - Biom) - BETA DIVERSITY ########################################

### CALCULATE BRAY CURTIS BETA DIVERSITY METRIC FOR TABLE ###
def table_beta_bray_curtis(bt):
    # GET BIOMTABLE COUNTS AS MATRIX WITH SAMPLES AS COLUMNS (TRANSFORM ROW<->COLS AND CONVERT TO INT)
    matrixDataAsInt = bt.matrix_data.todense().T.astype(int)
    # CALCULATE BETA DIVERSITY - BRAY CURTIS
    return sk.diversity.beta_diversity('braycurtis', matrixDataAsInt, ids=bt.ids(axis='sample')) 

### CALCULATE BINARY JACCARD BETA DIVERSITY METRIC FOR TABLE ###
def table_beta_binary_jaccard(bt):
    # GET TABLE AS PRESENCE / ABSENCE #
    bt = table_presence_absence(bt)
    # GET BIOMTABLE COUNTS AS MATRIX WITH SAMPLES AS COLUMNS (TRANSFORM ROW<->COLS AND CONVERT TO INT)
    matrixDataAsInt = bt.matrix_data.todense().T.astype(int)
    # CALCULATE BETA DIVERSITY - BRAY CURTIS
    return sk.diversity.beta_diversity('jaccard', matrixDataAsInt, ids=bt.ids(axis='sample')) 

### CALCULATE BINARY JACCARD BETA DIVERSITY METRIC FOR TABLE ###
def table_beta_unweighted_unifrac(bt, treeBetaIn):
    # GET BIOMTABLE COUNTS AS MATRIX WITH SAMPLES AS COLUMNS (TRANSFORM ROW<->COLS AND CONVERT TO INT)
    matrixDataAsInt = bt.matrix_data.todense().T.astype(int)
    # CALCULATE BETA DIVERSITY - BRAY CURTIS
    return sk.diversity.beta_diversity('unweighted_unifrac', matrixDataAsInt, ids=bt.ids(axis='sample'), 
                                       otu_ids=bt.ids(axis='observation'), tree=treeBetaIn) 

### CALCULATE BINARY JACCARD BETA DIVERSITY METRIC FOR TABLE ###
def table_beta_weighted_unifrac(bt, treeBetaIn):
    # GET BIOMTABLE COUNTS AS MATRIX WITH SAMPLES AS COLUMNS (TRANSFORM ROW<->COLS AND CONVERT TO INT)
    matrixDataAsInt = bt.matrix_data.todense().T.astype(int)
    # CALCULATE BETA DIVERSITY - BRAY CURTIS
    return sk.diversity.beta_diversity('weighted_unifrac', matrixDataAsInt, ids=bt.ids(axis='sample'), 
                                       otu_ids=bt.ids(axis='observation'), tree=treeBetaIn) 

### CALCULATE AN AVERAGE BETA DIVERSITY DISTANCE MATRIX FROM A LIST OF DATA MATRICES ###
def table_beta_avg_matrices(distanceMatrixList):
    ### STORAGE FOR AVERAGE DISTANCE MATRIX ###
    dmAvg = None
    ### FOR EACH DISTANCE MATRIX
    for idx, dmIncrement in enumerate(distanceMatrixList):
        ### GET NUMPY ARRAY OF FIRST DISTANCE MATRIX ###
        if idx == 0: dmAvg = dmIncrement.data
        ### ELSE ADD SUBSEQUENT MATRICES TO FIRST DISTANCE MATRIX ###
        else: dmAvg = dmAvg + dmIncrement.data
    ### RETURN DISTANCE MATRIX OBJECT DIVIDED BY THE NUMBER OF MATRICES ADDED TOGETHER ###
    return DistanceMatrix(dmAvg/(idx+1),dmIncrement.ids)

### UPGMA CLUSTERING ON BETA DIVERSITY DISTANCE MATRIX ###
def table_beta_upgma_cluster(betaDiversityMatrixIn):
    ### CALCULATE BETA DIVERSITY LINKAGE MATRIX ###
    betaLinkage = linkage(betaDiversityMatrixIn.condensed_form(), method='average')
    ### CALCULATE UPGMA TREE FROM LINKAGE MATRIX ###
    upgmaTree = TreeNode.from_linkage_matrix(betaLinkage, betaDiversityMatrixIn.ids)
    return upgmaTree

### CALCULATE CONSENSUS TREE FROM LIST OF UPGMA TREES ###
def table_beta_upgma_consensus(upgmaListIn): return sk.tree.majority_rule(upgmaListIn, weights=None, cutoff=0.5)

##### PIPELINE TO PERFORM BETA DIVERSITY CALCULATIONS #####
### THE INPUT BIOM TABLE bt WILL BE RAREFIED rarefactionsIn TIMES TO A DEPTH OF depthRareIn COUNTS PER SAMPLE ###
### BETA DIVERSITY METRIC TO USE: 'bray_curtis','binary_jaccard','unweighted_unifrac','weighted_unifrac'   
# UNIFRAC METRICS REQUIRE treeInIn BACTERIAL TREE
def table_beta_pipeline(rarefiedTablesIn, betaMetricIn='bray_curtis', treeInIn=None, saveFolder=None):
    print(" - Metric In: "+betaMetricIn)
    
    ### MAKE OUTPUT DIRECTORIES IF WRITING ###
    if saveFolder != None:
        if not os.path.isdir(saveFolder+"/"): os.makedirs(saveFolder+"/")
        if not os.path.isdir(saveFolder+"/rareDMs/"): os.makedirs(saveFolder+"/rareDMs/")
        if not os.path.isdir(saveFolder+"/rareUPGMAs/"): os.makedirs(saveFolder+"/rareUPGMAs/")

    ### CALCULATE BETA DIVERSITY DISTANCE MATRIX (DM) FOR EACH RAREFIED TABLE ###
    print(" - Creating List of Distance Matrices for Metric "+betaMetricIn+" -> rareDMs")
    rarefiedDMsIn = []
    for rareIdx, rareCycle in enumerate(rarefiedTablesIn): 
        
        if betaMetricIn == 'bray_curtis':
            dmCycleIn = beta_bray_curtis(rareCycle)
            rarefiedDMsIn.append(dmCycleIn)
            ### SAVE DISTANCE MATRICES ###
            if saveFolder != None: 
                if not os.path.isfile(saveFolder+"/rareDMs/dm_"+str(rareIdx)+".txt"):
                    dmCycleIn.write(saveFolder+"/rareDMs/dm_"+str(rareIdx)+".txt", format='lsmat')
            
        elif betaMetricIn == 'binary_jaccard':
            dmCycleIn = beta_binary_jaccard(rareCycle)
            rarefiedDMsIn.append(dmCycleIn)
            ### SAVE DISTANCE MATRICES ###
            if saveFolder != None: 
                if not os.path.isfile(saveFolder+"/rareDMs/dm_"+str(rareIdx)+".txt"):
                    dmCycleIn.write(saveFolder+"/rareDMs/dm_"+str(rareIdx)+".txt", format='lsmat')
                            
        elif betaMetricIn == 'unweighted_unifrac':
            dmCycleIn = beta_unweighted_unifrac(rareCycle, treeInIn)
            rarefiedDMsIn.append(dmCycleIn)
            ### SAVE DISTANCE MATRICES ###
            if saveFolder != None: 
                if not os.path.isfile(saveFolder+"/rareDMs/dm_"+str(rareIdx)+".txt"):
                    dmCycleIn.write(saveFolder+"/rareDMs/dm_"+str(rareIdx)+".txt", format='lsmat')
                 
        elif betaMetricIn == 'weighted_unifrac': 
            dmCycleIn = beta_weighted_unifrac(rareCycle, treeInIn)
            rarefiedDMsIn.append(dmCycleIn)
            ### SAVE DISTANCE MATRICES ###
            if saveFolder != None: 
                if not os.path.isfile(saveFolder+"/rareDMs/dm_"+str(rareIdx)+".txt"):
                    dmCycleIn.write(saveFolder+"/rareDMs/dm_"+str(rareIdx)+".txt", format='lsmat')
                 
        else: print("ERROR: Invalid Beta Metric!!!"); return
    ### CALCULATE AVERAGE DISTANCE MATRIX ###
    print(" - Calculating Average Distance Matrix -> consensusDM")
    consensusDMIn = beta_avg_matrices(rarefiedDMsIn)
    if saveFolder != None: consensusDMIn.write(saveFolder+"/consensus_dm.txt", format='lsmat')
    
    ### CALCULATE UPGMA TREE FOR EACH DM ###
    print(" - Creating List of UPGMA Trees for Each Distance Matrix -> rareUPGMAs")
    rarefiedUPGMAList = []
    for rareUPGMAidx, rareDMsCycle in enumerate(rarefiedDMsIn): 
        rareUPGMAIn = beta_upgma_cluster(rareDMsCycle)
        rarefiedUPGMAList.append(rareUPGMAIn)
        ### SAVE UPGMA TREES ###
        if saveFolder != None: 
                if not os.path.isfile(saveFolder+"/rareUPGMAs/upgma_"+str(rareUPGMAidx)+".tre"):
                    rareUPGMAIn.write(saveFolder+"/rareUPGMAs/upgma_"+str(rareUPGMAidx)+".tre", format='newick') 
        
    ### CALCULATE CONSENSUS UPGMA TREE ###
    print(" - Calculating Average UPGMA Tree -> consensusUPGMA")
    consensusUPGMAIn = beta_upgma_consensus(rarefiedUPGMAList)[0]
    if saveFolder != None: consensusUPGMAIn.write(saveFolder+"/consensus_upgma.tre", format='newick')
        
    ##### RETURN: LIST[RAREFIED_TABLES], LIST[RAREFIED_DMS], AVG_RAREFIED_DM, ListRarefiedUPGMA, consensusUPGMAIn
    return rarefiedDMsIn, consensusDMIn, rarefiedUPGMAList, consensusUPGMAIn

### BETA DIVERSITY ANALYSES ###
def table_beta_pipeline_analysis(consensusDMIn, mapDfIn, mapCatIn, displayIn=True, savePath=None):
    
    ######### SORT DISTANCES INTO DICT[GROUP1][GROUP2]=LIST #########
    valueDict = {}; print(" - Sorting Distances into Dictionary - ")
    sampleIndices = consensusDMIn.ids
    
    ### LOOP THROUGH DISTANCE MATRIX DIAGONALLY ###
    for sampleOnePos, sampleOne in enumerate(sampleIndices[:-1]):
        #if sampleOnePos%100==0: print(sampleOnePos)
        
        ### GET METADATA VALUE FOR SAMPLE AND CHECK DICTIONARY ###
        sampleOneMeta = mapDfIn.loc[sampleOne][mapCatIn]
        for sampleTwo in sampleIndices[(sampleOnePos+1):]:
            sampleTwoMeta = mapDfIn.loc[sampleTwo][mapCatIn]
            if hash(sampleOneMeta) < hash(sampleTwoMeta): 
                if sampleOneMeta not in valueDict.keys(): valueDict[sampleOneMeta] = {}
                if sampleTwoMeta not in valueDict[sampleOneMeta].keys(): valueDict[sampleOneMeta][sampleTwoMeta] = []
                valueDict[sampleOneMeta][sampleTwoMeta].append(consensusDMIn[sampleOne,sampleTwo])
            else: 
                if sampleTwoMeta not in valueDict.keys(): valueDict[sampleTwoMeta] = {}
                if sampleOneMeta not in valueDict[sampleTwoMeta].keys(): valueDict[sampleTwoMeta][sampleOneMeta] = []
                valueDict[sampleTwoMeta][sampleOneMeta].append(consensusDMIn[sampleOne,sampleTwo])
    
    ### COLLATE INTRA GROUP DISTANCES ###
    intra=[]; outList=[]; outList.append(" ----- INTRA-GROUP COMPARISONS ----- ")
    for group1 in valueDict.keys():
        outList.append(" - Intra: "+group1+" - ")
        outList.append("   Mean Diversity    : "+str(np.mean(valueDict[group1][group1])))
        outList.append("   Standard Deviation: "+str(np.std(valueDict[group1][group1])))
        outList.append("")
        intra.extend(valueDict[group1][group1])

    ### COLLATE INTER GROUP DISTANCES ###
    inter=[];outList.append(" ----- INTER-GROUP COMPARISONS ----- ")
    for group1 in valueDict.keys():
        for group2 in valueDict[group1].keys():
            if group1 != group2:
                outList.append(" - Inter: "+group1+" - "+group2+" - ")
                outList.append("   Mean Diversity    : "+str(np.mean(valueDict[group1][group2])))
                outList.append("   Standard Deviation: "+str(np.std(valueDict[group1][group2])))
                outList.append("")
                inter.extend(valueDict[group1][group2])

    ### ALL GROUPS TOGETHER ###
    outList.insert(0,"")
    outList.insert(0,"   Standard Deviation:" + str(np.std(inter)))
    outList.insert(0,"   Mean Diversity    :" + str(np.mean(inter)))
    outList.insert(0," - ALL INTER-GROUP -")
    outList.insert(0,"")
    outList.insert(0,"   Standard Deviation:" + str(np.std(intra)))
    outList.insert(0,"   Mean Diversity    :" + str(np.mean(intra)))
    outList.insert(0," - ALL INTRA-GROUP -")

    ######## STATISTICAL COMPARISONS ########
    outList.extend([" ----- STATISTICAL COMPARISONS -----",""," --- ALL GROUPS INTRA VS INTER --- ",""])
    outList.extend(list_mannwhitney(intra, inter, l1Name="All Intra-Group", l2Name="All Inter-Group", bonferroniComparisons=1)[0])
    outList.extend([""," --- COMPARISON OF INTRA-GROUP DISTANCES BETWEEN GROUPS --- ",""])


    ### INTRA-GROUP STATS ###
    bonCorrect = (((len(valueDict.keys())-1)*len(valueDict.keys()))/2)
    for group1 in valueDict.keys():
        for group2 in valueDict[group1].keys():
            if group1 != group2: 
                outList.extend(list_mannwhitney(valueDict[group1][group1], valueDict[group2][group2], l1Name=("Intra-"+group1), l2Name=("Intra-"+group2), bonferroniComparisons=bonCorrect)[0])
                outList.append("")

    ### INTRA-GROUP & INTER-GROUP STATS ###
    bonCorrect = ((len(valueDict.keys())-1)*len(valueDict.keys()))         
    outList.extend([""," --- COMPARISON OF INTRA-GROUP DISTANCE WITH INTER- TO EACH OTHER GROUP --- ",""])
    for group1 in valueDict.keys():
        for group2 in valueDict.keys():
            if group1 != group2:
                try: outList.extend(list_mannwhitney(valueDict[group1][group1], valueDict[group1][group2], l1Name=("Intra-"+group1), l2Name=("Inter-"+group1+"-"+group2), bonferroniComparisons=bonCorrect)[0])
                except: outList.extend(list_mannwhitney(valueDict[group1][group1], valueDict[group2][group1], l1Name=("Intra-"+group1), l2Name=("Inter-"+group2+"-"+group1), bonferroniComparisons=bonCorrect)[0])
                outList.append("")

    ### OUTPUT STATISTICS ###
    if displayIn == True: list_print(outList)
    if savePath != None: list_write(outList, savePath+".txt")
    
    ######## VIOLIN PLOT ########
    
    ##### COLLATE DISTANCES FOR VIOLIN PLOT #####
    violinIn = [intra,inter]; violinLabels = ["All Intra-Group", "All Inter-Group"]; violinType = [0,1] # 0intra, 1inter
    ### COLLATE INTER-INTRA ###
    for group1 in valueDict.keys():
        violinIn.append(valueDict[group1][group1]); violinLabels.append("Intra-"+group1); violinType.append(0)
        for group2 in valueDict.keys():
            if group1 != group2: 
                try:    violinIn.append(valueDict[group1][group2]); violinLabels.append("Inter-"+group1+"-"+group2); violinType.append(1)
                except: violinIn.append(valueDict[group2][group1]); violinLabels.append("Inter-"+group1+"-"+group2); violinType.append(1)

    ### GENERATE VIOLIN PLOT BY AWB ###
    fig, ax = plt.subplots(1,1, figsize=(20,10))
    f1 = ax.violinplot(violinIn,np.arange(len(violinIn)),showmeans=True)

    ### SET COLORS AND DIRECTIONALITY ###
    for idxType, vType in enumerate(violinType):
        if vType == 0:   b = f1['bodies'][idxType]; m = np.mean(b.get_paths()[0].vertices[:, 0]); b.get_paths()[0].vertices[:, 0] = np.clip(b.get_paths()[0].vertices[:, 0], -np.inf, m); b.set_color('r')
        elif vType == 1: b = f1['bodies'][idxType]; m = np.mean(b.get_paths()[0].vertices[:, 0]); b.get_paths()[0].vertices[:, 0] = np.clip(b.get_paths()[0].vertices[:, 0], m, np.inf); b.set_color('b')

    ### SET LABELS ###
    ax.set_xticks(np.arange(len(violinLabels)))
    ax.set_xticklabels(violinLabels, rotation=90)

    ### LABEL AND OUTPUT ###
    plt.title("Beta Diversity Distances")
    plt.xlim([-1,(len(violinLabels))])
    plt.tight_layout()
    if savePath != None: plt.savefig(savePath+".pdf")
    if displayIn == True: plt.show()
    plt.clf()
    return valueDict

### PERFORM ANOSIM TESTS ON LIST OF RAREFIED DISTANCE MATRICES AND COMPUTE AVERAGE STATISTICS ###
def table_beta_pipeline_anosim(rareDMsIn, mapDfIn, categoryIn, permutationsIn=99, savePath=None, displayIn=True):
    ### CALCULATE ANOSIM RESULTS FOR EACH DM AND STORE IN LIST ###
    testStatSum = []; pValSum = []; outList=[]
    ### LOOP THROUGH DISTANCE MATRICES ###
    outList.append(" - Calculating ANOSIM for Rarefied DMs Distinguishability by: "+categoryIn+" - ")
    for idx, rareDMCycle in enumerate(rareDMsIn):
        ### CALCULATE ANOSIM RESULTS ###
        anosimCycleResults = sk.stats.distance.anosim(rareDMCycle, mapDfIn, column=categoryIn, permutations=permutationsIn)
        ### ADD TEST STAT ###
        testStatSum.append(anosimCycleResults['test statistic'])
        ### ADD P-VALUE ###
        pValSum.append(anosimCycleResults['p-value'])
        ### SAVE VALUES ###
        if savePath != None: anosimCycleResults.to_csv(path=savePath+"_all.txt", index=True, 
                                                       sep='\t', na_rep='', header=True, mode='a')
    
    testStatAvg = np.mean(testStatSum); pValAvg = np.mean(pValSum)
    outList.append("   - Average ANOSIM R-stat : " + str(testStatAvg))
    outList.append("   - Average ANOSIM P-val  : " + str(pValAvg))
    outList.append("   - Sample Size           : " + str(anosimCycleResults['sample size']))
    outList.append("   - Number of Groups      : " + str(anosimCycleResults['number of groups']))
    outList.append("   - Number of Permutations: " + str(anosimCycleResults['number of permutations']))
    outList.append("")
    if displayIn == True: list_print(outList)
    if savePath != None: list_write(outList, savePath+".txt")
    """
    if (savePath != None) or (displayIn==True):
        try:
            ##### ANOSIM PLOTS OF RESULTING R AND P VALUES #####
            warnings.filterwarnings("ignore")

            ##### PLOT ANOSIM RESULTS #####
            fig, axs = plt.subplots(2,2,figsize=(12,6))

            ### PLOT R-VALUE DISTRIBUTION ###
            plt.subplot(2,2,1)
            sns.distplot(testStatSum, hist=True, kde=True, rug=True, bins=10, color="k")
            # LABEL PLOT #
            plt.title("ANOSIM R-value Distribution")
            plt.xlabel("R-value")
            # ADD MEAN LINE IN RED #
            ymin, ymax = plt.gca().get_ylim()
            plt.vlines(np.mean(testStatSum),ymin,ymax,color='r')
            # HIDE Y AXIS #
            plt.gca().get_yaxis().set_visible(False)

            ### PLOT R-VALUE DISTRIBUTION (0-1) ###
            plt.subplot(2,2,2)
            sns.distplot(testStatSum, hist=True, kde=True, rug=True, bins=10, color="k")
            # LABEL PLOT #
            plt.title("ANOSIM R-value Distribution (0-1)")
            plt.xlabel("R-value")
            plt.xlim([0,1])
            # ADD MEAN LINE IN RED #
            ymin, ymax = plt.gca().get_ylim()
            plt.vlines(np.mean(testStatSum),ymin,ymax,color='r')
            # HIDE Y AXIS #
            plt.gca().get_yaxis().set_visible(False)

            ### PLOT P-VALUE DISTRIBUTION ###
            plt.subplot(2,2,3)
            sns.distplot(pValSum, hist=True, kde=True, rug=True, bins=10, color="k")
            # LABEL PLOT #
            plt.title("ANOSIM P-value Distribution")
            plt.xlabel("P-value")
            # ADD MEAN LINE IN RED #
            ymin, ymax = plt.gca().get_ylim()
            plt.vlines(np.mean(pValSum),ymin,ymax,color='r')
            # HIDE Y AXIS #
            plt.gca().get_yaxis().set_visible(False)

            ### PLOT P-VALUE DISTRIBUTION (0-1) ###
            plt.subplot(2,2,4)
            sns.distplot(pValSum, hist=True, kde=True, rug=True, bins=10, color="k")
            # LABEL PLOT #
            plt.title("ANOSIM P-value Distribution (0-1)")
            plt.xlabel("P-value")
            plt.xlim([0,1])
            # ADD MEAN LINE IN RED #
            ymin, ymax = plt.gca().get_ylim()
            plt.vlines(np.mean(pValSum),ymin,ymax,color='r')
            # HIDE Y AXIS #
            plt.gca().get_yaxis().set_visible(False)
            plt.tight_layout()
            ### OUTPUT ###
            if savePath != None: plt.savefig(savePath+".pdf")
            if displayIn == True: plt.show()
            plt.clf()
        except ValueError:
            print("   Error Plotting ANOSIM!!!")
        warnings.filterwarnings("default")
    """
    return testStatAvg, pValAvg, testStatSum, pValSum

##### PERFORM ANOSIM ON ALL SUBSETS OF A METADATA GROUPING #####     
def table_beta_anosim_metadata_subsets(consensusDMIn, mapDfIn, mapCatIn, permutationsIn=99, savePath=None):
    ### DATAFRAME TO STORE RESULTS ###
    anosimDF = pd.DataFrame(data=None, index=None, columns=['test statistic','p-value'])
    ### GET SUBSETS OF DM ###
    dmCycler = dm_metadata_combinations(consensusDMIn, mapDfIn, mapCatIn)
    ### FOR EACH SET OF GROUPS ###
    for dmCycleIn in dmCycler:
        ### CALCULATE ANOSIM ###
        anosimResultsIn = sk.stats.distance.anosim(dmCycleIn[1], mapDfIn, column=mapCatIn, permutations=permutationsIn)
        ### STORE RESULTS ###
        anosimDF.loc[str(dmCycleIn[0])] = [anosimResultsIn['test statistic'],anosimResultsIn['p-value']]
        ### OUTPUT RESULTS ###
        anosimResultsIn[mapCatIn] = str(dmCycleIn[0])
        if savePath != None: anosimResultsIn.to_csv(path=savePath+"_table.txt", index=True, 
                                                       sep='\t', na_rep='', header=True, mode='a')
        
    return anosimDF

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h1 style="text-align:center; color:orange;"> - Load Data - </h1>

In [15]:
### LOAD A DATASET GIVEN PATHS ###
def load_dataset(dirPath=None,mapPathIn=None,tablePathIn=None,treePathIn=None, alphaPathIn=None):
    ### MAKE OUTPUT FOLDER ###
    print(" - Making Output Directory "+dirPath+" - ")
    if not os.path.isdir(dirPath): os.makedirs(dirPath)
    ### IMPORT MAPPING FILE ###
    if mapPathIn is not None:
        print(' - Importing Map: '+mapPathIn+' - ')
        mapDfOut = DF((mapPathIn))
        mapDfOut = mapDfOut.set_i(colNameOrList=["#SampleID"], append=False)
        mapDfOut = mapDfOut.df
    ### IMPORT BIOM TABLE ###
    if tablePathIn is not None:
        print(' - Importing Table:'+tablePathIn); biomTableOut = load_table(tablePathIn); print('   SUCCESS');
        ### GET TABLE METADATA ###
        metaCatsOut, metaDictOut = table_metadata(biomTableOut, mapDfOut)
        ### ADD METADATA TO BIOM TABLE ###
        biomTableOut.add_metadata(metaDictOut)
        ##### GET TABLE INFO #####
        otus,otusCounts,samples,samplesCounts = table_info(biomTableOut, writePath=None, printOut = True)
    ### IMPORT TREE ###
    if treePathIn is not None:
        try: print(' - Importing Tree: '+treePathIn); treeOut = tree_load(treePathIn); print('   SUCCESS')
        except: print('   FAILED')
    ##### LOAD ALPHA DIVERSITY #####
    if alphaPathIn is not None:
        try: print(' - Importing Alpha: '+alphaPathIn); alphaDf = pd.read_csv(alphaPathIn, sep='\t', index_col=0, skiprows=0, verbose=False, low_memory=False); print('   SUCCESS')
        except: print('   FAILED')
            
    return dirPath,mapDfOut,biomTableOut,treeOut,otus,otusCounts,samples,samplesCounts,metaCatsOut,metaDictOut,alphaDf

### ROTATE LOADING CONSENSUS DISTANCE MATRICES FOR VARYING DEPTHS AND BETA DIVERSITY METRICS ###
# RETURNS ConsensusIterator[curDepth, curMetric, consensusDMOut, rareDMIterator]
def dm_iter_consensus(dirPath, betaMetricsIn=['bray_curtis','unweighted_unifrac'],betaDepthsIn=[1000,10000]):
    ### IMPORT BETA ###
    for depIn in betaDepthsIn:
        for metIn in betaMetricsIn:
            ### LOAD CONSENSUS DISTANCE MATRICES ###
            print("importing: "+dirPath+'4_0_beta_diversity/4_0_beta_'+str(depIn)+'_'+metIn+'/consensus_dm.txt')
            conDMOut = DistanceMatrix.read(dirPath+'4_0_beta_diversity/4_0_beta_'+str(depIn)+'_'+metIn+'/consensus_dm.txt')
            ### ITERATOR FOR INDIVIDUAL RAREFIED DMS ###
            curRareDMs = dm_iter_folder(dirPath+'4_0_beta_diversity/4_0_beta_'+str(depIn)+'_'+metIn+'/rareDMs/*')
                
            yield depIn, metIn, conDMOut, curRareDMs


### ROTATES LOADING EACH DISTANCE MATRIX FROM A FOLDER ###
# INCORPORATES GLOB REGEX #
def dm_iter_folder(dirPath='dm_folder/*'): 
    for i in glob.glob(dirPath): yield DistanceMatrix.read(i)

        
### LOAD SINGULAR DATA STRUCTURES ###
dirPath,mapDf,biomTable,treeIn,otus,otusCounts,samples,samplesCounts,metaCats,metaDict,alphaDf = load_dataset(dirPath='test_data/ag_analysis/',
             mapPathIn='test_data/ag_analysis/1_1_qc_1000_map.txt',
             tablePathIn='test_data/ag_analysis/1_0_qc_1000_table/1_0_qc_1000_table.txt',
             treePathIn='test_data/ag_analysis/1_2_qc_1000_tree.tre',
             alphaPathIn='test_data/ag_analysis/3_0_alpha_diversity/3_0_alpha_diversity_1000.txt',
            ) 

### LOAD CYCLER FOR DISTANCE MATRICES UNDER VARYING DEPTHS AND METRICS ###
dmCycle = dm_iter_consensus(dirPath='test_data/ag_analysis/',betaMetricsIn=['bray_curtis','unweighted_unifrac'],betaDepthsIn=[1000,10000])

 - Making Output Directory test_data/ag_analysis/ - 
 - Importing Map: test_data/ag_analysis/1_1_qc_1000_map.txt - 
 - Loading DF File - test_data/ag_analysis/1_1_qc_1000_map.txt
 - SUCCESS: Loaded DF from: test_data/ag_analysis/1_1_qc_1000_map.txt - 
 - Importing Table:test_data/ag_analysis/1_0_qc_1000_table/1_0_qc_1000_table.txt
   SUCCESS
 - Getting Metadata Categories - 
 - Constructing Dictionary of Metadata by Samples - 
 - Table Info - 
   Total Observations: 5591
   Total Samples: 1375
   Total Counts: 29041078.0
   Non-Zero Entries: 616106
   Table Density: 0.08014256678753191
 - Importing Tree: test_data/ag_analysis/1_2_qc_1000_tree.tre
 - Loading Bacterial Phylogeny - test_data/ag_analysis/1_2_qc_1000_tree.tre
   SUCCESS
 - Importing Alpha: test_data/ag_analysis/3_0_alpha_diversity/3_0_alpha_diversity_1000.txt
   SUCCESS


<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h1 style="text-align:center; color:orange;"> - PlayPen - </h1>

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h2 style="text-align:center; color:blue;"> - ANOSIM Variants - </h2>

<h4 style="text-align:center; color:black;"> - Examine ANOSIM across varying depths and metrics - </h4>

In [16]:
##### PERFORM ANOSIM ON ALL SUBSETS OF A METADATA GROUPING #####     
def beta_anosim_metadata_subsets(consensusDMIn, mapDfIn, mapCatIn, permutationsIn=99, savePath=None):
    ### DATAFRAME TO STORE RESULTS ###
    anosimDF = pd.DataFrame(data=None, index=None, columns=['test statistic','p-value'])
    ### GET SUBSETS OF DM ###
    dmCycler = dm_metadata_combinations(consensusDMIn, mapDfIn, mapCatIn)
    ### FOR EACH SET OF GROUPS ###
    for dmCycleIn in dmCycler:
        ### CALCULATE ANOSIM ###
        anosimResultsIn = sk.stats.distance.anosim(dmCycleIn[1], mapDfIn, column=mapCatIn, permutations=permutationsIn)
        ### STORE RESULTS ###
        anosimDF.loc[str(dmCycleIn[0])] = [anosimResultsIn['test statistic'],anosimResultsIn['p-value']]
        ### OUTPUT RESULTS ###
        anosimResultsIn[mapCatIn] = str(dmCycleIn[0])
        if savePath != None: anosimResultsIn.to_csv(path=savePath+"_table.txt", index=True, 
                                                       sep='\t', na_rep='', header=True, mode='a')
    ### RETURN DATAFRAME OF RESULTS ###
    return anosimDF

<h4 style="text-align:center; color:black;"> - Examine ANOSIM across varying depths and metrics - </h4>

In [28]:
######## ANOSIM VARYING DEPTHS AND METRICS #######
##### IMPORT CONSENSUS DISTANCE MATRICES #####
# DISTANCE MATRICES WILL BE STORED IN A DICTIONARY consensusDMs[DEPTH][BETA_METRIC] = DISTANCE MATRIX
depthsIn = [1000, 10000]
metricsIn =['bray_curtis','binary_jaccard']#,'unweighted_unifrac','weighted_unifrac']
mapCat = 'race'

### EXTRACT DISTANCE MATRICES FROM FILES ###
consensusDMs = {}
for depIn in depthsIn:
    consensusDMs[depIn] = {}
    for metIn in metricsIn:
        print(' - Reading DM: '+str(depIn)+' '+metIn)
        consensusDMs[depIn][metIn] = DistanceMatrix.read('test_data/ag_analysis/4_0_beta_diversity/4_0_beta_'+str(depIn)+'_'+metIn+'/consensus_dm.txt')

##### PERFORM ANOSIM SUBSET ANALYSES FOR EACH DEPTH AND METRIC #####
anosimSubsets = {}
anosimList = []
### FOR EACH DEPTH ###
for depIn in consensusDMs.keys():
    anosimSubsets[depIn] = {}
    ### FOR EACH BETA METRIC ###
    for metIn in consensusDMs[depIn].keys():
        ### ANALYZE ANOSIM ON ALL SUBSETS OF A METADATA CATEGORY FOR DEPTH depIn AND BETA METRIC metIn
        print(' - Analyzing DM: '+str(depIn)+' '+metIn)
        anosimRes = beta_anosim_metadata_subsets(consensusDMs[depIn][metIn], mapDf, mapCat, permutationsIn=99, savePath=None)
        anosimRes['metric'] = metIn; anosimRes['depth']=depIn
        anosimSubsets[depIn][metIn] = anosimRes
        anosimList.append(anosimRes)

anosimAll = pd.concat(anosimList,axis=0)
anosimAll.set_index('depth',append=True, inplace=True)
anosimAll.set_index('metric',append=True, inplace=True)
display(anosimAll.sort_index())

 - Reading DM: 1000 bray_curtis
 - Reading DM: 1000 binary_jaccard
 - Reading DM: 10000 bray_curtis
 - Reading DM: 10000 binary_jaccard
 - Analyzing DM: 1000 binary_jaccard
 - All Table Combinations of 2 groups - 
   - Group: ('Caucasian', 'Hispanic')
   - Group: ('Caucasian', 'Asian or Pacific Islander')
   - Group: ('Caucasian', 'African American')
   - Group: ('Hispanic', 'Asian or Pacific Islander')
   - Group: ('Hispanic', 'African American')
   - Group: ('Asian or Pacific Islander', 'African American')
 - All Table Combinations of 3 groups - 
   - Group: ('Caucasian', 'Hispanic', 'Asian or Pacific Islander')
   - Group: ('Caucasian', 'Hispanic', 'African American')
   - Group: ('Caucasian', 'Asian or Pacific Islander', 'African American')
   - Group: ('Hispanic', 'Asian or Pacific Islander', 'African American')
 - All Table Combinations of 4 groups - 
   - Group: ('Caucasian', 'Hispanic', 'Asian or Pacific Islander', 'African American')
 - Analyzing DM: 1000 bray_curtis
 - All Ta

test statistic  \
                                                   depth metric                           
('Asian or Pacific Islander', 'African American')  1000  binary_jaccard        0.154271   
                                                         bray_curtis          -0.011845   
                                                   10000 binary_jaccard        0.233627   
                                                         bray_curtis          -0.004701   
('Caucasian', 'African American')                  1000  binary_jaccard        0.221099   
                                                         bray_curtis           0.081094   
                                                   10000 binary_jaccard        0.277874   
                                                         bray_curtis           0.096240   
('Caucasian', 'Asian or Pacific Islander')         1000  binary_jaccard        0.108984   
                                                         bray_curtis           0.157208   
                                                   10000 binary_jaccard        0.111655   
                                                         bray_curtis           0.161586   
('Caucasian', 'Asian or Pacific Islander', 'Afr... 1000  binary_jaccard        0.126655   
                                                         bray_curtis           0.146312   
                                                   10000 binary_jaccard        0.137943   
                                                         bray_curtis           0.152374   
('Caucasian', 'Hispanic')                          1000  binary_jaccard       -0.176563   
                                                         bray_curtis          -0.132173   
                                                   10000 binary_jaccard       -0.161706   
                                                         bray_curtis          -0.138136   
('Caucasian', 'Hispanic', 'African American')      1000  binary_jaccard       -0.052909   
                                                         bray_curtis          -0.066255   
                                                   10000 binary_jaccard       -0.024681   
                                                         bray_curtis          -0.065601   
('Caucasian', 'Hispanic', 'Asian or Pacific Isl... 1000  binary_jaccard        0.027325   
                                                         bray_curtis           0.075259   
                                                   10000 binary_jaccard        0.033464   
                                                         bray_curtis           0.076664   
('Caucasian', 'Hispanic', 'Asian or Pacific Isl... 1000  binary_jaccard        0.049993   
                                                         bray_curtis           0.076397   
                                                   10000 binary_jaccard        0.062304   
                                                         bray_curtis           0.079425   
('Hispanic', 'African American')                   1000  binary_jaccard        0.366344   
                                                         bray_curtis           0.234405   
                                                   10000 binary_jaccard        0.367661   
                                                         bray_curtis           0.251902   
('Hispanic', 'Asian or Pacific Islander')          1000  binary_jaccard       -0.053665   
                                                         bray_curtis          -0.103403   
                                                   10000 binary_jaccard       -0.032071   
                                                         bray_curtis          -0.110063   
('Hispanic', 'Asian or Pacific Islander', 'Afri... 1000  binary_jaccard        0.025353   
                                                         bray_curtis          -0.070532   
                                                   10000 binary_jacca

<h4 style="text-align:center; color:black;"> - Examine ANOSIM when SubSetting Individuals from Each Race - </h4>

In [22]:
######## ANOSIM VARYING DEPTHS AND METRICS #######
##### IMPORT CONSENSUS DISTANCE MATRICES #####
# DISTANCE MATRICES WILL BE STORED IN A DICTIONARY consensusDMs[DEPTH][BETA_METRIC] = DISTANCE MATRIX
depthsIn = [1000, 10000]
metricsIn =['bray_curtis','binary_jaccard']#,'unweighted_unifrac','weighted_unifrac']
mapCat = 'race'

### EXTRACT DISTANCE MATRICES FROM FILES ###
consensusDMs = {}
for depIn in depthsIn:
    consensusDMs[depIn] = {}
    for metIn in metricsIn:
        print(' - Reading DM: '+str(depIn)+' '+metIn)
        consensusDMs[depIn][metIn] = DistanceMatrix.read('test_data/ag_analysis/4_0_beta_diversity/4_0_beta_'+str(depIn)+'_'+metIn+'/consensus_dm.txt')

##### PERFORM ANOSIM SUBSET ANALYSES FOR EACH DEPTH AND METRIC #####
anosimSubsets = {}
anosimList = []

 - Reading DM: 1000 bray_curtis
 - Reading DM: 1000 binary_jaccard
 - Reading DM: 10000 bray_curtis
 - Reading DM: 10000 binary_jaccard


In [27]:

##### ANOSIM SUBSET INDIVIDUALS FROM GROUPS - DIFFERENTLY SAMPLED GROUPINGS #####
anosimTT = pd.DataFrame(data=None, index=None, columns=['test statistic','p-value', 'metric', 'depth', 'subsample', 'numsamples'])

minSamples = [10]#,15,20,25,30,35,40,45,50,75,100,250,500,1000,2000] 
subSampleNumber = 10


### PERFORM ANOSIM ON SUBSETS OF EACH RACE ###
for depIn in consensusDMs.keys():
    if depIn == 1000: mapDf = DF("test_data/ag_analysis/1_1_qc_1000_map.txt").df
    if depIn == 10000: mapDf = DF("test_data/ag_analysis/1_1_qc_10000_map.txt").df
    mapDf = mapDf.set_index('#SampleID')
    anosimSubsets[depIn] = {}
    for metIn in consensusDMs[depIn].keys():
        print(' - Analyzing DM: '+str(depIn)+' '+metIn)
        
        ### FOR EACH SAMPLING SIZE ###
        for minSam in minSamples:
            
            pvalsOut = [];rStatsOut=[]
            ### FOR THE NUMBER OF SUBSAMPLES ###
            for numSubSample in np.arange(subSampleNumber):
                ### FOR EACH ETHNICITY EXTRACT SAMPLES ###
                incSamples = []
                for eIn in mapDf['race'].unique():
                    if minSam <= mapDf[mapDf['race'] == eIn].shape[0]: 
                        incSamples.extend(mapDf[mapDf['race'] == eIn].sample(n=minSam, axis=0, replace=False,  random_state=54321).index)
                    else: incSamples.extend(mapDf[mapDf['race'] == eIn].index)
                ### TRIM DISTANCE MATRIX ###
                #print(incSamples); print()
                curDMFiltered = consensusDMs[depIn][metIn].filter(incSamples)
                anosimRes = sk.stats.distance.anosim(curDMFiltered, mapDf, column='race', permutations=99)
                pvalsOut.append(anosimRes['p-value']); rStatsOut.append(anosimRes['test statistic'])
            
            print(str(minSam), str(np.mean(rStatsOut)),str(np.mean(pvalsOut)))
            display(rStatsOut)
            display(pvalsOut)
            anosimTT.loc[str(depIn)+"_"+metIn+"_"+str(minSam)] = [np.mean(rStatsOut),np.mean(pvalsOut), metIn, depIn, minSam,len(incSamples)] 
    
    anosimTT.set_index('metric', append=True, inplace=True)
    anosimTT.set_index('depth', append=True, inplace=True)
    anosimTT.set_index('subsample', append=True, inplace=True)

 - Loading DF File - test_data/ag_analysis/1_1_qc_1000_map.txt
 - SUCCESS: Loaded DF from: test_data/ag_analysis/1_1_qc_1000_map.txt - 
 - Analyzing DM: 1000 binary_jaccard
10 0.0655740740741 0.042


[0.065574074074073993,
 0.065574074074073993,
 0.065574074074073993,
 0.065574074074073993,
 0.065574074074073993,
 0.065574074074073993,
 0.065574074074073993,
 0.065574074074073993,
 0.065574074074073993,
 0.065574074074073993]

[0.059999999999999998,
 0.029999999999999999,
 0.050000000000000003,
 0.050000000000000003,
 0.070000000000000007,
 0.029999999999999999,
 0.040000000000000001,
 0.040000000000000001,
 0.02,
 0.029999999999999999]

 - Analyzing DM: 1000 bray_curtis
10 0.0436666666667 0.092


[0.043666666666666742,
 0.043666666666666742,
 0.043666666666666742,
 0.043666666666666742,
 0.043666666666666742,
 0.043666666666666742,
 0.043666666666666742,
 0.043666666666666742,
 0.043666666666666742,
 0.043666666666666742]

[0.089999999999999997,
 0.089999999999999997,
 0.11,
 0.11,
 0.089999999999999997,
 0.080000000000000002,
 0.12,
 0.10000000000000001,
 0.050000000000000003,
 0.080000000000000002]

 - Loading DF File - test_data/ag_analysis/1_1_qc_10000_map.txt
 - SUCCESS: Loaded DF from: test_data/ag_analysis/1_1_qc_10000_map.txt - 
 - Analyzing DM: 10000 binary_jaccard
10 0.0824444444444 0.014


[0.082444444444444528,
 0.082444444444444528,
 0.082444444444444528,
 0.082444444444444528,
 0.082444444444444528,
 0.082444444444444528,
 0.082444444444444528,
 0.082444444444444528,
 0.082444444444444528,
 0.082444444444444528]

[0.02, 0.02, 0.01, 0.02, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02]

ValueError: cannot set a row with mismatched columns

In [26]:
mapDf

,#SampleID,race,sex,age_years,ageF,bmi,non_food_allergies_beestings,tonsils_removed,country_of_birth,pets_other,...,pku,state,age_corrected,mental_illness_type_unspecified,simple_body_site,title_acronym,title_body_site,hmp_site,age_group_custom,age_group_custom2
0,10317.000006703,Caucasian,male,54,54,27.47,False,Yes,United States,no_data,...,I do not have this condition,WV,54,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_40_55,Age_50_55
1,10317.000027821,Caucasian,male,25,25,25.54,False,No,United States,no_data,...,I do not have this condition,VA,25,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_18_29,Age_25_29
2,10317.000009594,Hispanic,female,52,52,20.70,False,Yes,United States,no_data,...,I do not have this condition,VA,52,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_40_55,Age_50_55
3,10317.000014543,Caucasian,female,46,46,21.05,False,Yes,United States,no_data,...,I do not have this condition,NC,46,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_40_55,Age_45_49
4,10317.000014548,Caucasian,male,39,39,35.17,False,Yes,United States,no_data,...,I do not have this condition,NJ,39,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_30_39,Age_35_39
5,10317.000028894,Caucasian,female,55,55,38.38,False,Yes,United States,no_data,...,I do not have this condition,TX,55,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_40_55,Age_50_55
6,10317.000009766,Caucasian,female,41,41,19.83,False,Yes,United States,no_data,...,Unknown,OH,41,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_40_55,Age_40_44
7,10317.000004738,Caucasian,male,37,37,21.46,False,Yes,United States,no_data,...,I do not have this condition,TX,37,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_30_39,Age_35_39
8,10317.000014608,Caucasian,female,30,30,17.62,False,No,United States,no_data,...,I do not have this condition,IL,30,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_30_39,Age_30_34
9,10317.000004734,Caucasian,female,31,31,23.38,False,No,United States,no_data,...,I do not have this condition,GA,31,no_data,FECAL,AGP,AGP-FECAL,FECAL,Age_30_39,Age_30_34


<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h2 style="text-align:center; color:blue;"> - Beta Diversity - Intra/Inter Comparison - </h2>

<h4 style="text-align:center; color:black;"> - beta_intra_inter(consensusDMIn, mapDfIn, mapCatIn, printOut=False): Compare beta diversity of every intra to inter category combination - </h4>

In [ ]:
### BETA DIVERSITY - INTRA/INTER DISTANCE COMPARISONS ###
# Calculate average distance from each sample to all individuals of other groups.
# input: consensus distance matrix, map, mapping file categorical variable
# output: pandas dataframes of p-values and distances
def beta_intra_inter(consensusDMIn, mapDfIn, mapCatIn, meanBool = True, printOut=False):
    
    
    # CONVERT TO DATAFRAME & GET INDEX #
    consDF = consensusDMIn.to_data_frame(); consIndex = consDF.index
    # ADD CATEGORY TO THE DISTANCE DATAFRAME #
    consDF = pd.concat([consDF, mapDfIn[mapCatIn]],axis=1)
    # ADD INDEX AS HASH TO DATAFRAME #
    hashindexOut = []
    for curIdx in consDF.index: hashindexOut.append(hash(curIdx))
    consDF['hashIndex'] = hashindexOut
    # GET UNIQUE GROUPS IN MAPPING CATEGORY #
    consCatGroups = list(mapDf[mapCatIn].unique())
    # MAKE DATAFRAME TO STORE RESULTS #
    betaIIDF = pd.DataFrame(data=None, index=consIndex, columns=(['sample', 'group']+consCatGroups))
    ### FOR EACH SAMPLE ###
    for sampleCycle in consIndex:
        ### STORE DISTANCES 
        groupDists = []
        ### GET MAPPING GROUP FOR SAMPLE ###
        curSampleGroup = mapDfIn[mapDfIn.index == sampleCycle][mapCatIn]
        
        ### STORE SAMPLE AND GROUP ###
        betaIIDF.loc[sampleCycle]['sample'] = sampleCycle
        betaIIDF.loc[sampleCycle]['group'] = curSampleGroup[0]

        ### FOR EACH GROUP ###
        for groupCycle in consCatGroups:

            ### GET DISTANCE MATRIX FOR ONLY GROUP ###
            curDM = consDF[(consDF[mapCatIn] == groupCycle)].copy()
            
            ### STORE LIST OF DISTANCES ###
            
            if meanBool == True: ### STORE AVERAGE SAMPLE DISTANCE TO SAMPLES IN GROUP ###
                ### REMOVE DIAGONAL DISTANCES == 0 ###
                #curDM = consDF[(consDF[mapCatIn] == groupCycle)].copy()
                curDM = curDM[curDM['hashIndex'] != hash(sampleCycle)][sampleCycle]
                betaIIDF.loc[sampleCycle][groupCycle] = curDM.mean() # STORE #
            
            else: ### STORE ALL DISTANCES TO SAMPLES IN OTHER GROUP ###
                ### REMOVE DIAGONAL DISTANCES AND DUPLICATE SAMPLING ###
                curDM = curDM[curDM['hashIndex'] > hash(sampleCycle)][sampleCycle]
                betaIIDF.loc[sampleCycle][groupCycle] = curDM.values # STORE #
        
    # MAKE DATAFRAME TO STORE RESULTS #
    betaDFOut = pd.DataFrame(data=None, index=None, columns=(['type','g0','g0dist','g1','g1dist','p-raw'])); idx=0
    
    
    ### FOR EACH COMBINATION OF GROUPS ###
    if printOut == True: print("- INTRA GROUP COMPARISONS - "); print()
    for combGroups in itertools.combinations(consCatGroups,2):
        
        ### GET FIRST GROUP AVERAGE DISTANCE ####
        if meanBool == True: intraDist0In = betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[0]].mean()
        else: intraDist0In = np.mean([item for sublist in betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[0]].values.tolist() for item in sublist])   
        if printOut == True:print(" - Average Intra Distance "+ combGroups[0]+" : "+ str(intraDist0In))
            
        ### GET SECOND GROUP AVERAGE DISTANCE ###
        if meanBool == True: intraDist1In = betaIIDF[betaIIDF['group'] == combGroups[1]][combGroups[1]].mean()
        else: intraDist1In = np.mean([item for sublist in betaIIDF[betaIIDF['group'] == combGroups[1]][combGroups[1]].values.tolist() for item in sublist])
        if printOut == True:print(" - Average Intra Distance "+ combGroups[1]+" : "+ str(intraDist1In))
            
        ### PERFORM MANN-WHITNEY-U TEST FIRST-SECOND ###
        if meanBool == True: resIn, tIn, pIn = stats_mannwhitney(betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[0]], betaIIDF[betaIIDF['group'] == combGroups[1]][combGroups[1]], l1Name=str("Intra-"+combGroups[0]), l2Name=str("Intra-"+combGroups[1]), bonferroniComparisons=len(list(itertools.combinations(consCatGroups,2))))
        else: resIn, tIn, pIn = stats_mannwhitney([item for sublist in betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[0]].values.tolist() for item in sublist],[item for sublist in betaIIDF[betaIIDF['group'] == combGroups[1]][combGroups[1]].values.tolist() for item in sublist],l1Name=str("Intra-"+combGroups[0]),l2Name=str("Intra-"+combGroups[1]),bonferroniComparisons=len(list(itertools.combinations(consCatGroups,2))))
        
        ### STORE RESULTS IN DATAFRAME ###
        betaDFOut.loc[idx] = ['Intra-Intra',combGroups[0],intraDist0In,combGroups[1],intraDist1In, pIn];idx+=1
        if printOut == True: list_print(resIn); print()

    
    ### FOR EACH COMBINATION OF GROUPS ###
    if printOut == True:print("- INTER GROUP COMPARISONS - "); print()
    for combGroups in itertools.permutations(consCatGroups,2):
       
        ### GET AVERAGE INTER DISTANCE ###
        if meanBool == True: interDist0In = betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[1]].mean()
        else: interDist0In = np.mean([item for sublist in betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[1]].values.tolist() for item in sublist])
        if printOut == True:print(" - Average Inter Distance "+ combGroups[0]+" to "+combGroups[1]+": "+ str(interDist0In))
            
        ### GET AVERAGE INTRA DISTANCE ###
        if meanBool == True: intraDist1In = betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[0]].mean()
        else: intraDist1In = np.mean([item for sublist in betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[0]].values.tolist() for item in sublist])
        if printOut == True:print(" - Average Intra Distance "+ combGroups[0]+": "+ str(intraDist1In))
            
        ### PERFORM MANN-WHITNEY-U TEST FIRST-SECOND ###
        if meanBool == True: resIn, tIn, pIn = stats_mannwhitney(betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[0]], betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[1]], l1Name=str("Intra-"+combGroups[0]), l2Name=str("Inter-"+combGroups[0]+"-"+combGroups[1]), bonferroniComparisons=len(list(itertools.permutations(consCatGroups,2))))
        else: resIn, tIn, pIn = stats_mannwhitney([item for sublist in betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[0]].values.tolist() for item in sublist],[item for sublist in betaIIDF[betaIIDF['group'] == combGroups[0]][combGroups[1]].values.tolist() for item in sublist],l1Name=str("Intra-"+combGroups[0]),l2Name=str("Inter-"+combGroups[0]+"-"+combGroups[1]),bonferroniComparisons=len(list(itertools.permutations(consCatGroups,2))))
        
        ### STORE RESULTS IN DATAFRAME ###
        betaDFOut.loc[idx] = ['Intra-Inter',combGroups[0],intraDist1In,combGroups[1],interDist0In, pIn];idx+=1
        if printOut == True: list_print(resIn); print()
    
    
    betaDFOut2 = pd.concat([betaDFOut,stats_pcorrect(betaDFOut['p-raw'],alphaIn=0.05)[['p-fdr','p-bonferroni']]],axis=1) 
    
    ### RETURN DATAFRAMES OF P-VALUES AND DISTANCES ###
    return betaDFOut2, betaIIDF

#consensusDM = i[2]
#betaIntraInterDist  = beta_intra_inter(consensusDM, mapDf, 'race', meanBool=False, printOut=True)

<h4 style="text-align:center; color:black;"> - Perform beta_intra_inter() on mean sample distances across depths and metrics and subsamples - </h4>

In [ ]:
##### INTRA/INTER ON MEAN SAMPLE DISTANCES #####

### CREATE CONSENSUS DM ITERABLE ###
dmCycle = dm_iter_consensus(dirPath='test_data/ag_analysis/',betaMetricsIn=['bray_curtis','weighted_unifrac','binary_jaccard','unweighted_unifrac'],betaDepthsIn=[1000,10000])
subSampleNumber = 10
minSamples = [10,15,20,25,30,35,40,45,50,75,100,250,500,1000,2000] ### MAXIMUM NUMBER OF SAMPLES FOR EACH RACE ###

curCols = ['type','g0','g1','depth','metric']+minSamples
outDM = DF(None, columnsIn=curCols)
outDict = {}
outDict2 = {}
mapCat = 'race'
outPath = 'test_data/ag_analysis/4_1_beta_intraInter/'
if not os.path.isdir(outPath): os.makedirs(outPath)
    
### FOR EACH CONSENSUS DM ###
for idx, i in enumerate(dmCycle):
    print(i[0],i[1])
    
    ### IMPORT APPROPRIATE MAP ###
    if i[0] == 1000: mapPathIn = 'test_data/ag_analysis/1_1_qc_1000_map.txt'
    else: mapPathIn = 'test_data/ag_analysis/1_1_qc_10000_map.txt'
    mapDf = DF((mapPathIn))
    mapDf = mapDf.set_i(colNameOrList=["#SampleID"], append=False)
    mapDf = mapDf.df

    ### FOR EACH SUBSAMPLE LEVEL ###
    for subLevel in minSamples:
        dfAvg = []
        print('   - Selecting Subset of '+str(subLevel)+' samples -')

        ### FOR THE NUMBER OF SUBSAMPLES ###
        for numSubSample in np.arange(subSampleNumber):
            
            ### FOR EACH GROUP EXTRACT SAMPLES ###
            incSamples = []
            for eIn in mapDf[mapCat].unique():
                ### IF THE NUMBER OF SAMPLES IS LARGER THAN THE CURRENT SUBSAMPLE 
                if subLevel < mapDf[mapDf[mapCat] == eIn].shape[0]: # THEN SELECT sublevel # OF SAMPLES
                    incSamples.extend(mapDf[mapDf[mapCat] == eIn].sample(n=subLevel, axis=0, replace=False).index)
                else: incSamples.extend(mapDf[mapDf[mapCat] == eIn].index)

            ### TRIM DISTANCE MATRIX ###
            curDMFiltered = i[2].filter(incSamples)
            
            ### PERFORM BETA ###
            iiInHere = beta_intra_inter(curDMFiltered, mapDf, mapCat, printOut=False)[0]
            
            ### FOR EACH COMPARISON GET TYPE AND DISTANCES ###
            for curIdx in iiInHere.index:
                typeIn = str(iiInHere.loc[curIdx]['type'])
                g0In = str(iiInHere.loc[curIdx]['g0'])
                g1In = str(iiInHere.loc[curIdx]['g1'])
            
                ### ADD DATA TO DICTIONARY ###
                if typeIn not in outDict.keys(): outDict[typeIn]={}
                if subLevel not in outDict[typeIn].keys(): outDict[typeIn][subLevel] = {}
                if g0In not in outDict[typeIn][subLevel].keys(): outDict[typeIn][subLevel][g0In] = {}
                if g1In not in outDict[typeIn][subLevel][g0In].keys(): outDict[typeIn][subLevel][g0In][g1In] = {}
                if i[1] not in outDict[typeIn][subLevel][g0In][g1In].keys(): outDict[typeIn][subLevel][g0In][g1In][i[1]] = {}
                if i[0] not in outDict[typeIn][subLevel][g0In][g1In][i[1]].keys(): outDict[typeIn][subLevel][g0In][g1In][i[1]][i[0]] = []
   
                ### STORE LIST OF P-VALUES
                outDict[typeIn][subLevel][g0In][g1In][i[1]][i[0]].append(iiInHere.loc[curIdx]['p-raw'])
                ### SAVE RESULTS FILE ###
                iiInHere.to_csv(outPath+'mean_'+[typeIn][0]+'_'+str(i[0])+'_'+[g0In][0].replace(' ','_')+'_'+[g1In][0].replace(' ','_')+'_'+[i[1]][0]+'_'+str([subLevel][0])+'.txt',sep='\t')
                    
### MAKE DATAFRAME FROM DICTIONARY ###
printOut = pd.DataFrame.from_dict({(i,j,k,l,m,n): outDict[i][j][k][l][m][n] 
                            for i in outDict.keys() 
                            for j in outDict[i].keys()
                            for k in outDict[i][j].keys()
                            for l in outDict[i][j][k].keys()
                            for m in outDict[i][j][k][l].keys()
                            for n in outDict[i][j][k][l][m].keys()}, orient='columns')

### SAVE OUTPUT TO TSV FILE ###
printOut = DF(printOut)
printOut.out_tsv(outPath="test_data/ag_analysis/4_1_beta_intraInter_mean_p_vals.txt") 
printEdit = printOut.copy()
printEdit.df.loc['Mean-p'] = printOut.df.mean(axis=0)
printEdit.df.loc['Median-p'] = printOut.df.median(axis=0)
printEdit.df.loc['Std-p'] = printOut.df.std(axis=0)
printEdit.df = printEdit.df.drop(printEdit.index[np.arange(10)])
printEdit.out_tsv(outPath="test_data/ag_analysis/4_1_beta_intraInter_mean_p_stats.txt") 


<h4 style="text-align:center; color:black;"> - Perform beta_intra_inter() on all sample distances across depths and metrics and subsamples - </h4>

In [ ]:
##### INTRA/INTER ON ALL PAIRWISE DISTANCES #####
### CREATE CONSENSUS DM ITERABLE ###
dmCycle = dm_iter_consensus(dirPath='test_data/ag_analysis/',betaMetricsIn=['bray_curtis','weighted_unifrac','binary_jaccard','unweighted_unifrac'],betaDepthsIn=[1000,10000])
subSampleNumber = 10
minSamples = [10,15,20,25,30,35,40,45,50,75,100,250,500,1000,2000] ### MAXIMUM NUMBER OF SAMPLES FOR EACH RACE ###

curCols = ['type','g0','g1','depth','metric']+minSamples
outDM = DF(None, columnsIn=curCols)
outDict = {}
outDict2 = {}
mapCat = 'race'
outPath = 'test_data/ag_analysis/4_1_beta_intraInter/'
if not os.path.isdir(outPath): os.makedirs(outPath)
    
### FOR EACH CONSENSUS DM ###
for idx, i in enumerate(dmCycle):
    print(i[0],i[1])
    
    ### IMPORT APPROPRIATE MAP ###
    if i[0] == 1000: mapPathIn = 'test_data/ag_analysis/1_1_qc_1000_map.txt'
    else: mapPathIn = 'test_data/ag_analysis/1_1_qc_10000_map.txt'
    mapDf = DF((mapPathIn))
    mapDf = mapDf.set_i(colNameOrList=["#SampleID"], append=False)
    mapDf = mapDf.df

    ### FOR EACH SUBSAMPLE LEVEL ###
    for subLevel in minSamples:
        dfAvg = []
        print('   - Selecting Subset of '+str(subLevel)+' samples -')

        ### FOR THE NUMBER OF SUBSAMPLES ###
        for numSubSample in np.arange(subSampleNumber):
            
            ### FOR EACH GROUP EXTRACT SAMPLES ###
            incSamples = []
            for eIn in mapDf[mapCat].unique():
                ### IF THE NUMBER OF SAMPLES IS LARGER THAN THE CURRENT SUBSAMPLE 
                if subLevel < mapDf[mapDf[mapCat] == eIn].shape[0]: # THEN SELECT sublevel # OF SAMPLES
                    incSamples.extend(mapDf[mapDf[mapCat] == eIn].sample(n=subLevel, axis=0, replace=False).index)
                else: incSamples.extend(mapDf[mapDf[mapCat] == eIn].index)

            ### TRIM DISTANCE MATRIX ###
            curDMFiltered = i[2].filter(incSamples)
            
            ### PERFORM BETA ###
            iiInHere = beta_intra_inter(curDMFiltered, mapDf, mapCat, meanBool=False, printOut=False)[0]
            
            ### FOR EACH COMPARISON GET TYPE AND DISTANCES ###
            for curIdx in iiInHere.index:
                typeIn = str(iiInHere.loc[curIdx]['type'])
                g0In = str(iiInHere.loc[curIdx]['g0'])
                g1In = str(iiInHere.loc[curIdx]['g1'])
            
                ### ADD DATA TO DICTIONARY ###
                if typeIn not in outDict.keys(): outDict[typeIn]={}
                if subLevel not in outDict[typeIn].keys(): outDict[typeIn][subLevel] = {}
                if g0In not in outDict[typeIn][subLevel].keys(): outDict[typeIn][subLevel][g0In] = {}
                if g1In not in outDict[typeIn][subLevel][g0In].keys(): outDict[typeIn][subLevel][g0In][g1In] = {}
                if i[1] not in outDict[typeIn][subLevel][g0In][g1In].keys(): outDict[typeIn][subLevel][g0In][g1In][i[1]] = {}
                if i[0] not in outDict[typeIn][subLevel][g0In][g1In][i[1]].keys(): outDict[typeIn][subLevel][g0In][g1In][i[1]][i[0]] = []
   
                ### STORE LIST OF P-VALUES
                outDict[typeIn][subLevel][g0In][g1In][i[1]][i[0]].append(iiInHere.loc[curIdx]['p-raw'])
                ### SAVE RESULTS FILE ###
                iiInHere.to_csv(outPath+'nonmean_'+[typeIn][0]+'_'+str(i[0])+'_'+[g0In][0].replace(' ','_')+'_'+[g1In][0].replace(' ','_')+'_'+[i[1]][0]+'_'+str([subLevel][0])+'_nonmean.txt',sep='\t')
                    
### MAKE DATAFRAME FROM DICTIONARY ###
printOut = pd.DataFrame.from_dict({(i,j,k,l,m,n): outDict[i][j][k][l][m][n] 
                            for i in outDict.keys() 
                            for j in outDict[i].keys()
                            for k in outDict[i][j].keys()
                            for l in outDict[i][j][k].keys()
                            for m in outDict[i][j][k][l].keys()
                            for n in outDict[i][j][k][l][m].keys()}, orient='columns')

### SAVE OUTPUT TO TSV FILE ###
printOut = DF(printOut)
printOut.out_tsv(outPath="test_data/ag_analysis/4_1_beta_intraInter_nonmean_p_vals.txt") 
printEdit = printOut.copy()
printEdit.df.loc['Mean-p'] = printOut.df.mean(axis=0)
printEdit.df.loc['Median-p'] = printOut.df.median(axis=0)
printEdit.df.loc['Std-p'] = printOut.df.std(axis=0)
printEdit.df = printEdit.df.drop(printEdit.index[np.arange(10)])
printEdit.out_tsv(outPath="test_data/ag_analysis/4_1_beta_intraInter_nonmean_p_stats.txt") 


In [ ]:
#kf = sklearn.model_selection.KFold(n_splits=3, shuffle=False, random_state=54321)

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h2 style="text-align:center; color:blue;"> - BioEnv - </h2>

In [ ]:
### CORRELATION BETWEEN DISTANCE MATRIX AND COVARIATE DATAFRAME FROM REGRESSION STYLE EQUATION
# TEST MODIFIED TO MEASURE CORRELATION OF ONLY SPECIFIED EQUATION, NOT ALL
# POSSIBLE SUBSETS OF VARIABLES
# ----------------------------------------------------------------------------
# Copyright (c) 2013--, scikit-bio development team.
#
# Distributed under the terms of the Modified BSD License.
#
# The full license is in the file COPYING.txt, distributed with this software.
# ----------------------------------------------------------------------------
def beta_bioenv(dmIn, mapDfIn, regEquation="[ANY CONTINUOUS COLUMN] ~ age + sex:race + bmi", columns=None):
    ### EXTRACT REGRESSION EQUATION FROM COVARIATES DATAFRAME ###
    y, mapDfIn = patsy.dmatrices(regEquation, mapDfIn, return_type='dataframe')
    ### MAKE SURE DISTACE MATRIX IS OBJECT ###
    if not isinstance(dmIn, DistanceMatrix): raise TypeError("Must provide a DistanceMatrix as input.")
    ### MAKE SURE MAP IS PANDAS DF ###
    if not isinstance(mapDfIn, pd.DataFrame): raise TypeError("Must provide a pandas.DataFrame as input.")  
    ### IF NO LIST OF COLUMNS TO USE THEN USE ALL ###
    columns = mapDfIn.columns.values.tolist()
    ### CHECK FOR DUPLICATE COLUMNS ###
    if len(set(columns)) != len(columns): raise ValueError("Duplicate column names are not supported.")
    ### CHECK TO MAKE SURE AT LEAST ON COLUMN FACTOR ###
    if len(columns) < 1: raise ValueError("Must provide at least one column.")
    ### MAKE SURE COLUMNS ARE IN DATAFRAME ###
    for column in columns:
        if column not in mapDfIn: raise ValueError("Column '%s' not in data frame." % column)
    ### ORDER COVARIATES TO SAMPLES IN DISTANCE MATRIX ###
    variablesDF = mapDfIn.loc[dmIn.ids, columns]
    ### CHECK FOR MISSING IDS IN DF FROM DM ###
    if variablesDF.isnull().any().any(): raise ValueError("One or more IDs in the distance matrix are not in the data frame, or there is missing data in the data frame.")
    ### CHECK ALL COVARIATES ARE CASTABLE AS FLOATS ##
    try: vars_df = vars_df.astype(float)
    except ValueError: raise TypeError("All specified columns in the data frame must be numeric.")
    ##### CENTER DATAFRAME AROUND MEAN OF EACH COVARIATE AND COMPUTE AS STD #####
    # Modified from http://stackoverflow.com/a/18005745
    dfCenterScale = variablesDF.copy()
    dfCenterScale -= dfCenterScale.mean()
    dfCenterScale /= dfCenterScale.std()
    ### MAKE SURE COULD BE CENTERED AROUND STD ###
    if dfCenterScale.isnull().any().any(): raise ValueError("Column(s) in the data frame could not be scaled, likely because the column(s) had no variance.")
    ### CAST AS ARRAY ###
    dfCovariatesArray = dfCenterScale.values
    ##### FLATTEN THE DISTANCE MATRIX #####
    dmInFlattened = dmIn.condensed_form()
    ### COMPUTE PREDICTOR DISTANCE MATRIX ###
    dfEuclidianDistance = sp.spatial.distance.pdist(dfCovariatesArray,metric='euclidean')
    ### CALCULATE CORRELATION BETWEEN DISTANCE MATRIX AND COVARIATE DISTANCES ###
    rhoOut = sp.stats.spearmanr(dmInFlattened, dfEuclidianDistance)[0]
    return rhoOut
    

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h2 style="text-align:center; color:blue;"> - PCoA - </h2>

In [ ]:
def pcoa(btIn, mapDfIn, regEquation):
    pcoaBiom = sk.stats.ordination.pcoa(dmIn, mapDfIn)
    Emperor(pcoaBiom, mapDf)

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h2 style="text-align:center; color:blue;"> - CCA - </h2>

In [ ]:
def cca(btIn, mapDfIn, regEquation):
    ### GET DATAFRAME OF RELATIVE BIOM TABLE COUNTS ###
    taIn = table_dataframe(table_relative(btIn))
    
    predy, covX = patsy.dmatrices(regEquation, mapDfIn, return_type='dataframe')
    del covX['Intercept']
    ccaBiom = sk.stats.ordination.cca(taIn, X)
    Emperor(ccaBiom, mapDf)


In [ ]:

### METHOD ###
ccaBiom.long_method_name

### EIGEN-VALUES ###
#ccaBiom.eigvals

### SAMPLE LOCATIONS ###
#ccaBiom.samples

### OTU LOCATIONS ###
#ccaBiom.features

### CONSTRAINING COVARIATES ###
ccaBiom.biplot_scores


#ccaBiom.sample_constraints

#ccaBiom.proportion_explained

In [ ]:
##### COMPARISONS OF EACH GROUP #####
# SHOULD ALL PAIRS
singleBoolIn = True
pairBoolIn = True

numGroupsIn = len(mapDf['race'].unique())
dmResults = sk.stats.distance.DistanceMatrix(data=np.zeros((numGroupsIn,numGroupsIn)), ids=mapDf['race'].unique())
dmResults = dmResults.to_data_frame()
allResOut = pd.DataFrame(columns=['size','correlation'])
##### IF ALL GROUPS CONTRASTED TO MEAN #####
if singleBoolIn == True:
    ### CREATE DISTANCE MATRIX OF PREDICTORS ###
    y, X = patsy.dmatrices(" age_years ~ age_years + sex + bmi ", mapDf, return_type='dataframe')
    ### REMOVE INTERCEPT COLUMN ###
    del X['Intercept']
    ### FOR EACH GROUP ###
    for i in mapDf['race'].unique():
        X[('race_'+i)] = np.where(mapCopy[('race')]==i, 0.75, -0.25)
        ### DISPLAY CORRELATION ALONE ###
        resIn = sk.stats.distance.bioenv(consensusDM, pd.DataFrame(data=[X['race_'+i]], index=['race_'+i]).T)
        allResOut = pd.concat([allResOut, resIn],axis=0)
        dmResults[i][i] = resIn['correlation'][0]
    ### PERFORM CORRELATION ANALYSIS WITH ALL ###
    print(list(X.columns))
    display(sk.stats.distance.bioenv(consensusDM, X))
    del X['age_years']
    del X['bmi']
    del X['sex[T.male]']
    print(list(X.columns))
    display(sk.stats.distance.bioenv(consensusDM, X))
    
##### IF ALL PAIRS OF GROUPS CONTRASTED #####
if pairBoolIn == True:
    ### CREATE DISTANCE MATRIX OF PREDICTORS ###
    y, X = patsy.dmatrices(" age_years ~ age_years + sex + bmi ", mapDf, return_type='dataframe')
    ### REMOVE INTERCEPT COLUMN ###
    del X['Intercept']
    ### FOR EACH PAIR OF GROUPS SHOW INDIVIDUAL CORRELATION ###
    for i in list(itertools.combinations(mapDf['race'].unique(),2)):
        X[('race_'+i[0]+'_'+i[1])] = np.where(mapCopy[('race')]==i[0], 1, 0)
        X[('race_'+i[0]+'_'+i[1])].ix[np.where(mapCopy[('race')]==i[1])] = -1
        resIn = sk.stats.distance.bioenv(consensusDM, pd.DataFrame(data=[X[('race_'+i[0]+'_'+i[1])]], index=[('race_'+i[0]+'_'+i[1])]).T)
        allResOut = pd.concat([allResOut, resIn],axis=0)
        dmResults[i[0]][i[1]] = resIn['correlation'][0]
        dmResults[i[1]][i[0]] = resIn['correlation'][0]
    ### PERFORM CORRELATION ANALYSIS WITH ALL ###
    print(list(X.columns))
    display(sk.stats.distance.bioenv(consensusDM, X))
    del X['age_years']
    del X['bmi']
    del X['sex[T.male]']
    print(list(X.columns))
    display(sk.stats.distance.bioenv(consensusDM, X))
    
    

sns.heatmap(dmResults)
plt.show()
display(allResOut)

In [ ]:
import patsy
import statsmodels.api as sm

### CALCULATE A DICTIONARY OF INFO ABOUT OTUS ###
def observation_info(bt, mapDfIn, mapCatsIn=None):
    print(" - COLLATING OBSERVATION INFORMATION -")
    ### CREATE DICTIONARY FOR INFO WITH TAXA FILLED###
    obsDfCounts = table_dataframe(bt).T
    obsDfTaxa = table_dataframe_taxa(bt)
    
    ##### ANALYSES OF OTUS ##########
    obsDfTaxa['OTU Mean'] = pd.Series(obsDfCounts.apply(np.mean, axis=1), index=obsDfCounts.index)
    obsDfTaxa['OTU Std All'] = pd.Series(obsDfCounts.apply(lambda x: np.std(x), axis=1), index=obsDfCounts.index)
    obsDfTaxa['NNZ'] = pd.Series(obsDfCounts.apply(lambda x: np.count_nonzero(x), axis=1), index=obsDfCounts.index)
    obsDfTaxa['NNZ Mean'] = pd.Series(obsDfCounts[obsDfCounts > 0].apply(np.mean, axis=1), index=obsDfCounts.index)
    return obsDfTaxa, obsDfCounts

def observation_correlation(mapDfIn, obsDfTaxaIn, obsDfCountsIn, independentEquation="col1 + col2 + col1:col2", zeroReplace='skip'):
    print(" - PERFORMING OBSERVATION REGRESSIONS -")
    print("   - OTU Abundance = "+independentEquation)
    
    ### FORM REGRESSION DATAFRAME ###
    inDFConcat = pd.concat([mapDfIn, obsDfCountsIn.T],axis=1)
    
    ### DATA STRUCTURE FOR RESULTS ###
    y, X = patsy.dmatrices("patsy.builtins.Q('"+obsDfTaxaIn.index[0]+"') ~ "+independentEquation, inDFConcat, return_type='dataframe')    
    colsIn = ['rsquared','f_pvalue']; colsIn.extend(X.columns)
    storeRegRes = pd.DataFrame(data=None,columns=colsIn,index=list(obsDfTaxaIn.index))

    ### HOW TO DEAL WITH ZERO VALUES ###
    #if zeroReplace != None: obsDfCountsIn = obsDfCountsIn.replace(0.0, zeroReplace)

    ### FOR EACH OTU ###
    for obsIdx, obsCycle in enumerate(obsDfCounts.index):
        
        ### Y = INDX... + B ||| SETUP INPUT MATRIX & CODE CATEGORICAL FACTORS ###
        y, X = patsy.dmatrices("np.log10(patsy.builtins.Q('"+str(obsCycle)+"')) ~ "+independentEquation, inDFConcat[inDFConcat[obsCycle] > 0], return_type='dataframe')
    
        ### GENERATE & FIT MODEL ###
        statsModelReg = sm.OLS(y, X)
        statsModelRes = statsModelReg.fit()

        ### COLLATE AND ADD RESULTS TO DATAFRAME ###
        storeRegRes.set_value(str(obsCycle), 'rsquared', statsModelRes.rsquared)
        storeRegRes.set_value(str(obsCycle), 'f_pvalue', statsModelRes.f_pvalue)
        for statIdx, statCur in enumerate(statsModelRes.pvalues.keys()): 
            storeRegRes.set_value(str(obsCycle), statCur, statsModelRes.pvalues.values[statIdx])
        
        ### PRINT PROGRESS ###
        if obsIdx %1000 == 0: 
            print("     - Progress: "+str(obsIdx))
    
    return pd.concat([obsDfTaxaIn, storeRegRes],axis=1)


### CALCULATE OTU INFO ###
obsDfTaxa, obsDfCounts = observation_info(table_relative(biomTable), mapDf, 'race')

### PERFORM REGRESSION AGAINST METADATA ###
#obsDfRegRes = observation_correlation(mapDf, obsDfTaxa, obsDfCounts, independentEquation="age_years + bmi + race + sex + race:sex", zeroReplace=None)
obsDfRegRes = observation_correlation(mapDf, obsDfTaxa, obsDfCounts, independentEquation=" race ", zeroReplace=None)

display(obsDfRegRes.sort_values(by='f_pvalue', ascending=True))
#display(obsDfRegRes.groupby([obsDfTaxa['phylum'],obsDfTaxa['class'],obsDfTaxa['order'],obsDfTaxa['family'],obsDfTaxa['genus']], ).mean())

In [ ]:
x1 = obsDfCounts.T['325850']


sns.distplot(np.log10(x1[x1 > 0.0]), bins=20)

In [ ]:
x1 = obsDfCounts.T['325850']


sns.distplot(np.arcsin(np.sqrt(x1[x1>0.0])), bins=20)

In [ ]:
pd.options.display.max_columns = None

In [ ]:
sm.graphics.plot_partregress('bmi', 'age_years', ['sex', 'race'], data=pd.concat([mapDf, obsDfCounts.T],axis=1), obs_labels=False)
plt.show()

In [ ]:
##### ANCOM PIPELINE - UNFUNCTION ATM #####
"""
def pipeline_ancom(bt, mapDfIn):
    ### TABLE AS RELATIVE ABUNDANCE ###
    btRelative = table_relative(bt)
    ### CONVERT TO PANDAS DATAFRAME (COLS = OTUS) ###
    btRelativePd = counts_as_pandas(btRelative)
    ### USE MULTIPLICITIVE REPLACEMENT TO REPLACE 0'S WITH 10^-8 VALUES ###
    btRelativePdRep = sk.stats.composition.multiplicative_replacement(btRelativePd, delta=None)
    ### CONVERT BACK TO PANDAS DF ###
    btRelativePdRepPd = pd.DataFrame(btRelativePdRep, index=bt.ids(axis='sample'), columns=bt.ids(axis='observation'))
    return sk.stats.composition.ancom(btRelativePdRepPd, 
                               mapDfIn, 
                               alpha=0.05, tau=0.02, theta=0.1, 
                               multiple_comparisons_correction='holm-bonferroni', 
                               significance_test=sp.stats.kruskal, 
                               percentiles=(0.0, 25.0, 50.0, 75.0, 100.0))

#x = pipeline_ancom(filter_otu_mincount(biomTable, 10), mapDf['race'])
"""

In [ ]:
sk.diversity.get_alpha_diversity_metrics()

In [ ]:
sk.diversity.get_alpha_diversity_metrics()

<h3 style="text-align:center; color:black;">------------------------------------------------------------------------------</h3>
<h2 style="text-align:center; color:orange;"> - Custom - </h2>

In [ ]:
# betaMetrics = ['braycurtis', 'jaccard', 'weighted_unifrac','unweighted_unifrac']
#http://scikit-bio.org/docs/0.1.3/generated/skbio.core.distance.DistanceMatrix.html#skbio.core.distance.DistanceMatrix 
#sk.core.distance.DistanceMatrix.filter(ids)
#sk.core.distance.DistanceMatrix.to_file(out_f[, delimiter])
#sk.core.distance.DistanceMatrix.permute([condensed])
#sk.core.distance.DistanceMatrix.copy()
#sk.core.distance.DistanceMatrix.condensed_form()

In [ ]:
### Gneiss Log ratio Abudnances toolkit

import gneiss

<h4 style="text-align:center; color:blue;"> - Partition by Experiment (Host Clade) - </h4>

In [ ]:
firstRun = False # Change this to True if you want to rewrite output to folder

### METADATA CATEGORY TO PARTITION BY ###
metaPartCat = "Experiment"

### PARTITION TABLE BY EXPERIMENT ###
expTables = partition_meta(biomTable, metaPartCat, print_summary=False)

### FOR EACH PARTITION...
for expName in expTables.keys():
    
    ### MAKE SUBDIRECTORY ###
    if firstRun == True: 
        if not os.path.isdir(dirPath+"0_2_"+expName): os.makedirs(dirPath+"0_2_"+expName)

    print()
    ### PRINT EXPERIMENT NAME ###
    print(expName)
    
    ### FILTER OUT OTUS WITH <2 COUNTS ###
    expTables[expName] = filter_otu_mincount(expTables[expName], 2)
    
    ### PRINT EXPERIMENT INFO ###
    if firstRun == True: get_table_info(expTables[expName], writePath=dirPath+"0_2_"+expName+"/"+"0_2_table_summary_"+expName+".txt", printOut = True)
    else: get_table_info(expTables[expName], printOut = True)

    ### PRINT COUNTS PER SAMPLE ###
    print("Sample Counts:  "+str(expTables[expName].sum('sample')))
    print("Minimum Sample: "+str(min(expTables[expName].sum('sample')))) 
    
    ### ADD METADATA TO EXPERIMENT TABLE ###
    expTables[expName].add_metadata(metaDict)
    
    ### WRITE TABLE AS TSV AND JSON ###
    if firstRun == True: write_tsv(expTables[expName], dirPath+"0_2_"+expName+""+"0_2_table_"+expName+".txt")
    if firstRun == True: write_json(expTables[expName], dirPath+"0_2_"+expName+"/"+"0_2_table_"+expName+".biom")

<h4 style="text-align:center; color:blue;"> - Perform Rarefactions within Each Clade 100x to Depth: (Minimum Sample-1) - </h4>

In [ ]:
### CREATE DICTIONARY TO HOLD RAREFIED TABLES ###
expTablesRarefied = {}
### FOR EACH PARTITION...
for expName in expTables.keys():
    ### CREATE ARRAY OF 100 RAREFIED TABLES AT A DEPTH OF (THE MINIMUM SAMPLE -1) ###
    print("Rarefying: "+expName+" to a depth of "+str(int(min(expTables[expName].sum('sample')) - 1)))
    expTablesRarefied[expName] = multiple_rarefactions_even_depth(expTables[expName], int(min(expTables[expName].sum('sample')) - 1), 100)

<h4 style="text-align:center; color:blue;"> - Calculate Beta Diversity for the 100 Rarefied Tables of Each Clade - </h4>

In [ ]:
### CREATE DICTIONARY TO HOLD BETA DIVERSITY DISTANCE MATRICES
expTablesBC = {}
### FOR EACH PARTITION...
for expName in expTablesRarefied.keys():
    print("Calculating Beta Diversity: " + expName); expTablesBC[expName] = []
    ### FOR EACH RAREFIED TABLE
    for rareTableIncrement in expTablesRarefied[expName]:
        expTablesBC[expName].append(beta_braycurtis(rareTableIncrement))

In [ ]:

for expName in expTablesBC.keys():
    print(expName)
    ### CALCULATE CONSENSUS UPGMA TREE ###
    print(beta_upgma_consensus(expTablesBC[expName])[0].ascii_art())


In [ ]:
for i in dmAvgDict.keys():
    dmAvgDict[i].plot()

In [ ]:
dmAvg


In [ ]:



# write output
fTreeOut = open(output_file, 'w')
fTreeOut.write(upgmaTree.to_newick(with_distances=True))
fTreeOut.close()


    
# SciPy uses average as UPGMA:
# http://docs.scipy.org/doc/scipy/reference/generated/
# scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage


    

#!/usr/bin/env python

__author__ = "Justin Kuczynski"
__copyright__ = "Copyright 2011, The QIIME Project"
__credits__ = ["Justin Kuczynski"]
__license__ = "GPL"
__version__ = "1.9.1"
__maintainer__ = "Justin Kuczynski"
__email__ = "justinak@gmail.com"

"""runs upgma or nj on a distance matrix file, writes the resulting tree
with distances to specified output file
"""
import warnings
warnings.filterwarnings('ignore', 'Not using MPI as mpi4py not found')
from optparse import OptionParser
from qiime.parse import parse_distmat
from  import linkage
from  import TreeNode
from skbio.tree import nj
import os.path


def multiple_file_upgma(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    file_names = os.listdir(input_dir)
    file_names = [fname for fname in file_names if not fname.startswith('.')]

    for fname in file_names:
        base_fname, ext = os.path.splitext(fname)
        single_file_upgma(os.path.join(input_dir, fname),
                          os.path.join(output_dir, 'upgma_' + base_fname + '.tre'))


def single_file_upgma(input_file, output_file):
    # read in dist matrix
    

    


def single_file_nj(input_file, output_file):
    dm = DistanceMatrix.read(input_file)

    tree = nj(dm)

    # write output
    f = open(output_file, 'w')
    f.write(tree.to_newick(with_distances=True))
    f.close()


def multiple_file_nj(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    file_names = os.listdir(input_dir)
    file_names = [fname for fname in file_names if not fname.startswith('.')]

    for fname in file_names:
        base_fname, ext = os.path.splitext(fname)
        single_file_nj(os.path.join(input_dir, fname),
                       os.path.join(output_dir, 'nj_' + base_fname + '.tre'))

<h3 style="text-align:center; color:black;">------------------------------------------------------------------------------</h3>
<h3 style="text-align:center; color:orange;"> - IMPORT FAMILY LEVEL RELATIVE ABUNDANCE BIOM TABLE - </h3>

In [ ]:
famPath = "/Users/brooks/Dropbox/American_Gut/Data/21_0_summarize_taxa/20_2_filter_country_L5.txt"
famTable = load_table(famPath)

<h3 style="text-align:center; color:orange;"> - EXTRACT CHRISTENSENELLACEAE & ADD TO MAPPING FILE - </h3>

In [ ]:
### ITERATE OVER OTUS - EXTRACT CHRISTENSENELLACEAE ###
iterOTUs = famTable.iter(axis='observation')
for values, id, metadata in iterOTUs:
    if id == "k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Christensenellaceae":
        print id
        break

mapChrist = pd.concat([mapDf, pd.DataFrame(data=values, index=get_samples(famTable), columns=['Christ'])], axis=1)

<h3 style="text-align:center; color:orange;"> - AVERAGE BMI WITH AND WITHOUT CHRISTENSENELLACEAE - </h3>

In [ ]:
### AVERAGE BMI'S ###

mapAllWith = mapChrist[mapChrist['Christ'] > 0]
mapAllWithout = mapChrist[mapChrist['Christ'] == 0]

### ALL INDIVIDUALS ###
print "Number of Individuals :" + str(len(mapChrist))
print "Average BMI: " + str(mapChrist['bmi'].mean())
print

### ALL WITH CHRISTENSENELLACEAE ###
print "ALL WITH CHRISTENSENELLACEAE"
print "Number of Individuals :" + str(len(mapAllWith))
print "Average BMI: " + str(mapAllWith['bmi'].mean())
print

### ALL WITHOUT CHRISTENSENELLACEAE ###
print "ALL WITHOUT CHRISTENSENELLACEAE"
print "Number of Individuals :" + str(len(mapAllWithout))
print "Average BMI: " + str(mapAllWithout['bmi'].mean())
print

##################################################################################
### Function - Mann-Whitney U Test - Nonparametric rank test of lists
# Null hypothesis: two samples from the same population 
# Alternative hypothesis: one population tends to have larger values than the other [Wikipedia]
# N samples is > 20 and you have 2 independent samples of ranks (can be unequal lengths) [Scipy]
# For two tailed test multiply P-Value*2
import scipy.stats as sp
# IN: Two independent lists of floats
# OUT: Mann Whitney Test Statistic and P-Value
def list_mannwhitney(l1, l2, outFile=None, bonferroniComparisons=1):
    # use_continuity = Whether a continuity correction (1/2.) should be taken into account. Default is True. [Scipy]
    outMann = sp.mannwhitneyu(l1, l2, use_continuity=True)
    print "Mann Whitney U - Nonparametric Rank Test"
    if outFile != None: outFile.write("Mann Whitney U - Nonparametric Rank Test" + "\n")
    print "    List #1 Length: "+str(len(l1))+" | List #2 Length: "+str(len(l2))
    if outFile != None: outFile.write("    List #1 Length: "+str(len(l1))+" | List #2 Length: "+str(len(l2)) + "\n")
    print "    Test Statistic: "+str(outMann[0])
    if outFile != None: outFile.write("    Test Statistic: "+str(outMann[0]) + "\n")
    print "    P-Value (onetailed): "+str(outMann[1])
    if outFile != None: outFile.write("    P-Value (onetailed): "+str(outMann[1]) + "\n")
    print "    P-Value (twotailed): "+str(outMann[1]*2)
    if outFile != None: outFile.write("    P-Value (twotailed): "+str(outMann[1]*2) + "\n")
    
    print "    P-Value Bonferroni Corrected (twotailed): "+str((outMann[1]*2)*bonferroniComparisons)
    if outFile != None: outFile.write("    P-Value Bonferroni Corrected (twotailed): "+str((outMann[1]*2)*bonferroniComparisons) + "\n")
    print
    return outMann
##################################################################################

### COMPARE THOSE WITH TO THOSE WITHOUT USING LIST_MANNWHITNEY ###
list_mannwhitney(mapAllWith['bmi'], mapAllWithout['bmi'])

<h3 style="text-align:center; color:orange;"> - UBIQUITY OF CHRISTENSENELLACEAE - </h3>

In [ ]:
### GET UBIQUITY BY RACE - ISOLATE RACES AS SEPARATE TABLES ###
mapAA = mapChrist[mapChrist['race'] == "African American"]
mapAS = mapChrist[mapChrist['race'] == "Asian or Pacific Islander"]
mapCA = mapChrist[mapChrist['race'] == "Caucasian"]
mapHI = mapChrist[mapChrist['race'] == "Hispanic"]

#### UBIQUITY FUNCTION ####
def ubiquityCount(dfIn):
    print "Non-Zero Individuals      : " + str(dfIn['Christ'].astype(bool).sum())
    print "Total Individuals         : " + str(len(dfIn['Christ']))
    print "Ubiquity                  : " + str( float(dfIn['Christ'].astype(bool).sum()) / float(len(dfIn['Christ'])))
    print 

# CALCULATE UBIQUITY #
print "AA"
ubiquityCount(mapAA)

print "AS"
ubiquityCount(mapAS)

print "CA"
ubiquityCount(mapCA)

print "HI"
ubiquityCount(mapHI)


<h3 style="text-align:center; color:orange;"> - AVERAGE BMI AND CHRISTENSENELLACEAE BY RACE - </h3>
<h4 style="text-align:center; color:orange;"> - This includes the boxplots generated for figure 4 - </h4>
<h4 style="text-align:center; color:orange;"> - and the statistics of BMI for those with and without Christensenellaceae used in figure 4 - </h4>
<h4 style="text-align:center; color:orange;"> - Also includes some additional stats and an additional regression figure not used in manuscript - </h4>

In [ ]:
### GET AVERAGE BMI AND CHRISTENSENELLACEAE FOR EACH RACE - INCLUDING FOR THOSE WITH AND WITHOUT PRESENCE SUBSET ###
# Get log10 transformation of counts
mapChrist["ChristLog"] = np.log10(mapChrist["Christ"])
# Get arcsin sqrt transformation of counts (see Structure, Function, and Diversity of the Healthy Human Microbiome)
mapChrist["ChristArcSin"] = np.arcsin(np.sqrt(mapChrist["Christ"]))

print "GET AVERAGE BMI AND CHRISTENSENELLACEAE FOR EACH RACE - INCLUDING FOR THOSE WITH AND WITHOUT PRESENCE SUBSET"
# FOR ALL INDIVIDUALS
print 'AA Mean BMI                : ' +str(np.mean(mapChrist[mapChrist['race'] == 'African American']['bmi'] ))
print 'AA Mean Christensenellaceae: ' +str(np.mean(mapChrist[mapChrist['race'] == 'African American']['Christ'] ))
print
print 'AS Mean BMI                : ' +str(np.mean(mapChrist[mapChrist['race'] == 'Asian or Pacific Islander']['bmi'] ))
print 'AS Mean Christensenellaceae: ' +str(np.mean(mapChrist[mapChrist['race'] == 'Asian or Pacific Islander']['Christ'] ))
print
print 'CA Mean BMI                : ' +str(np.mean(mapChrist[mapChrist['race'] == 'Caucasian']['bmi'] ))
print 'CA Mean Christensenellaceae: ' +str(np.mean(mapChrist[mapChrist['race'] == 'Caucasian']['Christ'] ))
print
print 'HI Mean BMI                : ' +str(np.mean(mapChrist[mapChrist['race'] == 'Hispanic']['bmi'] ))
print 'HI Mean Christensenellaceae: ' +str(np.mean(mapChrist[mapChrist['race'] == 'Hispanic']['Christ'] ))

# Remove samples w/ no Christ
mapChristR = mapChrist[mapChrist["Christ"] != 0]
# FOR ONLY INDIVIDUALS WITH CHRISTENSENELLACEAE #
print
print '-----------------------------------------------'
print 'AA Mean BMI                : ' +str(np.mean(mapChristR[mapChristR['race'] == 'African American']['bmi'] ))
print 'AA Mean Christensenellaceae: ' +str(np.mean(mapChristR[mapChristR['race'] == 'African American']['Christ'] ))
print
print 'AS Mean BMI                : ' +str(np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['bmi'] ))
print 'AS Mean Christensenellaceae: ' +str(np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['Christ'] ))
print
print 'CA Mean BMI                : ' +str(np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['bmi'] ))
print 'CA Mean Christensenellaceae: ' +str(np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['Christ'] ))
print
print 'HI Mean BMI                : ' +str(np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['bmi'] ))
print 'HI Mean Christensenellaceae: ' +str(np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['Christ'] ))

# Remove samples w/ Christ
mapChristRWO = mapChrist[mapChrist["Christ"] == 0]
# FOR ONLY INDIVIDUALS WITHOUT CHRISTENSENELLACEAE #
print
print '-----------------------------------------------'
print 'AA Mean BMI                : ' +str(np.mean(mapChristRWO[mapChristRWO['race'] == 'African American']['bmi'] ))
print 'AA Mean Christensenellaceae: ' +str(np.mean(mapChristRWO[mapChristRWO['race'] == 'African American']['Christ'] ))
print 
print 'AS Mean BMI                : ' +str(np.mean(mapChristRWO[mapChristRWO['race'] == 'Asian or Pacific Islander']['bmi'] ))
print 'AS Mean Christensenellaceae: ' +str(np.mean(mapChristRWO[mapChristRWO['race'] == 'Asian or Pacific Islander']['Christ'] ))
print
print 'CA Mean BMI                : ' +str(np.mean(mapChristRWO[mapChristRWO['race'] == 'Caucasian']['bmi'] ))
print 'CA Mean Christensenellaceae: ' +str(np.mean(mapChristRWO[mapChristRWO['race'] == 'Caucasian']['Christ'] ))
print
print 'HI Mean BMI                : ' +str(np.mean(mapChristRWO[mapChristRWO['race'] == 'Hispanic']['bmi'] ))
print 'HI Mean Christensenellaceae: ' +str(np.mean(mapChristRWO[mapChristRWO['race'] == 'Hispanic']['Christ'] ))

##################################################################################
##################################################################################
##################################################################################

### COMPARE WITH AND WITHOUT BMIs - EXTRACT THOSE WITH AND WITHOUT CHRISTENSENELLACEAE FOR EACH RACE ###
# EACH RACE WITH CHRISTENSENELLACEAE #
mapAAr = mapChristR[mapChristR['race'] == "African American"]
mapASr = mapChristR[mapChristR['race'] == "Asian or Pacific Islander"]
mapCAr = mapChristR[mapChristR['race'] == "Caucasian"]
mapHIr = mapChristR[mapChristR['race'] == "Hispanic"]
# EACH RACE WITHOUT CHRISTENSENELLACEAE #
mapAArwo = mapChristRWO[mapChristRWO['race'] == "African American"]
mapASrwo = mapChristRWO[mapChristRWO['race'] == "Asian or Pacific Islander"]
mapCArwo = mapChristRWO[mapChristRWO['race'] == "Caucasian"]
mapHIrwo = mapChristRWO[mapChristRWO['race'] == "Hispanic"]

##################################################################################
### Function - Mann-Whitney U Test - Nonparametric rank test of lists
# Null hypothesis: two samples from the same population 
# Alternative hypothesis: one population tends to have larger values than the other [Wikipedia]
# N samples is > 20 and you have 2 independent samples of ranks (can be unequal lengths) [Scipy]
# For two tailed test multiply P-Value*2
import scipy.stats as sp
# IN: Two independent lists of floats
# OUT: Mann Whitney Test Statistic and P-Value
def list_mannwhitney(l1, l2, outFile=None, bonferroniComparisons=1):
    # use_continuity = Whether a continuity correction (1/2.) should be taken into account. Default is True. [Scipy]
    outMann = sp.mannwhitneyu(l1, l2, use_continuity=True)
    print "Mann Whitney U - Nonparametric Rank Test"
    if outFile != None: outFile.write("Mann Whitney U - Nonparametric Rank Test" + "\n")
    print "    List #1 Length: "+str(len(l1))+" | List #2 Length: "+str(len(l2))
    if outFile != None: outFile.write("    List #1 Length: "+str(len(l1))+" | List #2 Length: "+str(len(l2)) + "\n")
    print "    Test Statistic: "+str(outMann[0])
    if outFile != None: outFile.write("    Test Statistic: "+str(outMann[0]) + "\n")
    print "    P-Value (onetailed): "+str(outMann[1])
    if outFile != None: outFile.write("    P-Value (onetailed): "+str(outMann[1]) + "\n")
    print "    P-Value (twotailed): "+str(outMann[1]*2)
    if outFile != None: outFile.write("    P-Value (twotailed): "+str(outMann[1]*2) + "\n")
    
    print "    P-Value Bonferroni Corrected (twotailed): "+str((outMann[1]*2)*bonferroniComparisons)
    if outFile != None: outFile.write("    P-Value Bonferroni Corrected (twotailed): "+str((outMann[1]*2)*bonferroniComparisons) + "\n")
    print
    return outMann
##################################################################################

### COMPARE BMIS OF THOSE WITH CHRISTENSENELLACEAE TO THOSE WITHOUT FOR EACH RACE USING MANN WHITNEY U TEST ###
print "COMPARE BMIS OF THOSE WITH CHRISTENSENELLACEAE TO THOSE WITHOUT FOR EACH RACE USING MANN WHITNEY U TEST"
print "COMPARE EACH RACES BMI WITH AND WITHOUT CHRISTENSENELLACEAE #"
print
print "AA"
list_mannwhitney(mapAAr['bmi'], mapAArwo['bmi'])

print
print "AS"
list_mannwhitney(mapASr['bmi'], mapASrwo['bmi'])

print
print "CA"
list_mannwhitney(mapCAr['bmi'], mapCArwo['bmi'])

print
print "HI"
list_mannwhitney(mapHIr['bmi'], mapHIrwo['bmi'])
print

##################################################################################
##################################################################################
##################################################################################

import scipy
#### PLOT OF AVERAGES - THIS IS A SCATTER PLOT OF THE AVERAGE BMI BY THE AVERAGE CHRISTENSENELLACEAE FOR THOSE WITH AND WITHOUT THE PRESENECE OF CHRISTENSENELLACEAE ####
#### NOT USED IN MANUSCRIPT - BUT HAS REGRESSIONS LINES AND REGRESSIONS STATISTICS OUTPUT WHICH ARE REALLY ONLY INTERESTING TO EVALUATE SLOPE (P-VALS USELESS) 
plt.figure(figsize=[10,5])
print "PLOT OF AVERAGES - THIS IS A SCATTER PLOT OF THE AVERAGE BMI BY THE AVERAGE CHRISTENSENELLACEAE FOR THOSE WITH AND WITHOUT THE PRESENECE OF CHRISTENSENELLACEAE"
print "NOT USED IN MANUSCRIPT - BUT HAS REGRESSIONS LINES AND REGRESSIONS STATISTICS OUTPUT WHICH ARE REALLY ONLY INTERESTING TO EVALUATE SLOPE (P-VALS USELESS)"
# PLOT ALL INDIVIDUALS #
plt.scatter(np.mean(mapChrist[mapChrist['race'] == 'African American']['bmi'] ),np.mean(mapChrist[mapChrist['race'] == 'African American']['Christ'] ), c='y', s=100, marker='^')
plt.scatter(np.mean(mapChrist[mapChrist['race'] == 'Asian or Pacific Islander']['bmi'] ),np.mean(mapChrist[mapChrist['race'] == 'Asian or Pacific Islander']['Christ'] ), c='b', s=100, marker='^')
plt.scatter(np.mean(mapChrist[mapChrist['race'] == 'Caucasian']['bmi'] ),np.mean(mapChrist[mapChrist['race'] == 'Caucasian']['Christ'] ), c='r', s=100, marker='^')
plt.scatter(np.mean(mapChrist[mapChrist['race'] == 'Hispanic']['bmi'] ),np.mean(mapChrist[mapChrist['race'] == 'Hispanic']['Christ'] ), c='g', s=100, marker='^')

# PLOT ALL INDIVIDUALS WITH #
plt.scatter(np.mean(mapChristR[mapChristR['race'] == 'African American']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'African American']['Christ'] ), c='y', s=100, marker='>')
plt.scatter(np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['Christ'] ), c='b', s=100, marker='>')
plt.scatter(np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['Christ'] ), c='r', s=100, marker='>')
plt.scatter(np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['Christ'] ), c='g', s=100, marker='>')

# PLOT ALL INDIVIDUALS WITH #
plt.scatter(np.mean(mapChristRWO[mapChristRWO['race'] == 'African American']['bmi'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'African American']['Christ'] ), c='y', s=100, marker='*')
plt.scatter(np.mean(mapChristRWO[mapChristRWO['race'] == 'Asian or Pacific Islander']['bmi'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Asian or Pacific Islander']['Christ'] ), c='b', s=100, marker='*')
plt.scatter(np.mean(mapChristRWO[mapChristRWO['race'] == 'Caucasian']['bmi'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Caucasian']['Christ'] ), c='r', s=100, marker='*')
plt.scatter(np.mean(mapChristRWO[mapChristRWO['race'] == 'Hispanic']['bmi'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Hispanic']['Christ'] ), c='g', s=100, marker='*')

# REGRESSION ACROSS RACES OF INDIVIDIUALS WITH 
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChristR[mapChristR['race'] == 'African American']['bmi'] ), np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['bmi'] )],
    [np.mean(mapChristR[mapChristR['race'] == 'African American']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['Christ'] ),np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['Christ'] ),np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['Christ'] )])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'k--')
print
print "All With Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

# REGRESSION ACROSS RACES OF INDIVIDIUALS WITH TRANSFORMED
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChristR[mapChristR['race'] == 'African American']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['Christ'] ),np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['Christ'] ),np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['Christ'] )],
    [np.mean(mapChristR[mapChristR['race'] == 'African American']['bmi'] ), np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['bmi'] )])

print
print "All With Regression Transformed"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

# ADD REGRESSION LINES #
# AA
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChrist[mapChrist['race'] == 'African American']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'African American']['bmi'] ), np.mean(mapChristRWO[mapChristRWO['race'] == 'African American']['bmi'] ) ],
    [np.mean(mapChrist[mapChrist['race'] == 'African American']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'African American']['Christ'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'African American']['Christ'] ) ])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'y--')
print
print "African American Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

# AS
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChrist[mapChrist['race'] == 'Asian or Pacific Islander']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['bmi'] ), np.mean(mapChristRWO[mapChristRWO['race'] == 'Asian or Pacific Islander']['bmi'] ) ],
    [np.mean(mapChrist[mapChrist['race'] == 'Asian or Pacific Islander']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['Christ'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Asian or Pacific Islander']['Christ'] ) ])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'b--')
print
print "Asian or Pacific Islander Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

# CA
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChrist[mapChrist['race'] == 'Caucasian']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['bmi'] ), np.mean(mapChristRWO[mapChristRWO['race'] == 'Caucasian']['bmi'] ) ],
    [np.mean(mapChrist[mapChrist['race'] == 'Caucasian']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['Christ'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Caucasian']['Christ'] ) ])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'r--')
print
print "Caucasian Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

# HI
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChrist[mapChrist['race'] == 'Hispanic']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['bmi'] ), np.mean(mapChristRWO[mapChristRWO['race'] == 'Hispanic']['bmi'] ) ],
    [np.mean(mapChrist[mapChrist['race'] == 'Hispanic']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['Christ'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Hispanic']['Christ'] ) ])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'g--')
print
print "Hispanic Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

##########
plt.xlabel("BMI")
plt.ylabel("Relative Christensenellaceae")
plt.title("Christensenellaceae by BMI for Each Race - All Individuals & with / without")
plt.tight_layout()
plt.ylim(-0.001, 0.007)
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/AVG_PLOT2.pdf")
plt.show()


### PRINT REGRESSION TRANSFORMED ###
# ADD REGRESSION LINES #
print "NOT USED IN MANUSCRIPT - REGRESSIONS STATISTICS FOR TRANSFORMED CHRISTENSENELLACEAE ABUNDANCE OUTPUT WHICH ARE REALLY ONLY INTERESTING TO EVALUATE SLOPE (P-VALS USELESS)"

# AA
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChrist[mapChrist['race'] == 'African American']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'African American']['Christ'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'African American']['Christ'] ) ],
    [np.mean(mapChrist[mapChrist['race'] == 'African American']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'African American']['bmi'] ), np.mean(mapChristRWO[mapChristRWO['race'] == 'African American']['bmi'] ) ]

)
print
print "African American Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

# AS
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChrist[mapChrist['race'] == 'Asian or Pacific Islander']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['Christ'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Asian or Pacific Islander']['Christ'] ) ],
    [np.mean(mapChrist[mapChrist['race'] == 'Asian or Pacific Islander']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Asian or Pacific Islander']['bmi'] ), np.mean(mapChristRWO[mapChristRWO['race'] == 'Asian or Pacific Islander']['bmi'] ) ]
)
print
print "Asian or Pacific Islander Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

# CA
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(    
    [np.mean(mapChrist[mapChrist['race'] == 'Caucasian']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['Christ'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Caucasian']['Christ'] ) ],
    [np.mean(mapChrist[mapChrist['race'] == 'Caucasian']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Caucasian']['bmi'] ), np.mean(mapChristRWO[mapChristRWO['race'] == 'Caucasian']['bmi'] ) ]

)
print
print "Caucasian Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)

# HI
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(
    [np.mean(mapChrist[mapChrist['race'] == 'Hispanic']['Christ'] ), np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['Christ'] ),np.mean(mapChristRWO[mapChristRWO['race'] == 'Hispanic']['Christ'] ) ],
    [np.mean(mapChrist[mapChrist['race'] == 'Hispanic']['bmi'] ),np.mean(mapChristR[mapChristR['race'] == 'Hispanic']['bmi'] ), np.mean(mapChristRWO[mapChristRWO['race'] == 'Hispanic']['bmi'] ) ]
)
print
print "Hispanic Regression"
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)


### BOXPLOTS OF BMIS FOR INDIVDIDUALS WITH AND WITHOUT CHRISTENSENELLACEAE USED IN MANUSCRIPT - MEANS ARE INCLUDED AS RED BOXES ###
print "BOXPLOTS OF BMIS FOR INDIVDIDUALS WITH AND WITHOUT CHRISTENSENELLACEAE USED IN MANUSCRIPT - MEANS ARE INCLUDED AS RED BOXES"
ax1 = mapChrist.boxplot(column='bmi', by='race', figsize=[15,8], showmeans=True)
plt.title('BMI of All Individuals')
ax1.set_ylim(0,60)
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/bmi_all.pdf")

ax2 = mapChristR.boxplot(column='bmi', by='race', figsize=[15,8], showmeans=True)
plt.title('BMI With Christensenellaceae')
ax2.set_ylim(0,60)
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/bmi_with.pdf")

ax3 = mapChristRWO.boxplot(column='bmi', by='race', figsize=[15,8], showmeans=True)
plt.title('BMI Without Christensenellaceae')
ax3.set_ylim(0,60)
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/bmi_without.pdf")

ax5 = mapChristR.boxplot(column='ChristLog', by='race', figsize=[15,8], showmeans=True)
plt.title('Christensenellaceae Log10 With Only')
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/log_abund.pdf")
ax5.set_ylim(-5,0)

ax5 = mapChrist.boxplot(column='ChristLog', by='race', figsize=[15,8], showmeans=True)
plt.title('Christensenellaceae Log10 All')
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/log_abund_all.pdf")
ax5.set_ylim(-5,0)

ax6 = mapChristR.boxplot(column='ChristArcSin', by='race', figsize=[15,8], showmeans=True)
plt.title('Christensenellaceae ArcSin Sqrt With Only')
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/arcsin_sqrt_abund.pdf")


<h3 style="text-align:center; color:orange;"> - REGRESSION OF BMI AGAINST CHRISTENSENELLACEAE ABUNDANCE - </h3>
<h4 style="text-align:center; color:orange;"> - These are regressions only including individuals with Christensenellaceae - </h4>

In [ ]:
# Get log10 transformation of counts
mapChrist["ChristLog"] = np.log10(mapChrist["Christ"])
# Get arcsin sqrt transformation of counts (see Structure, Function, and Diversity of the Healthy Human Microbiome)
mapChrist["ChristArcSin"] = np.arcsin(np.sqrt(mapChrist["Christ"]))

# Set Colors
colors = {'African American':'red', 'Asian or Pacific Islander':'blue', 'Caucasian':'green', 'Hispanic':'yellow'}

# Remove samples w/ no Christ
mapChrist = mapChrist[mapChrist["Christ"] != 0]

# Plot ArcSin
mapChrist.plot(kind='scatter', x='bmi', y='ChristArcSin', c=mapChrist['race'].apply(lambda x: colors[x]), figsize=[10,10])
plt.title("ArcSin sqrt Transformation")

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(mapChrist["bmi"],mapChrist["ChristArcSin"])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'r--')
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/reg_ArcSin_Chr_only.pdf")
plt.show()

# Plot Log10
mapChrist.plot(kind='scatter', x='bmi', y='ChristLog', c=mapChrist['race'].apply(lambda x: colors[x]), figsize=[10,10])
plt.title("Log10 Transformation")

mapChrist["ChristLog"] = mapChrist["ChristLog"].replace(to_replace='-inf', value=-10)

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(mapChrist["bmi"],mapChrist["ChristLog"])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'r--')
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/reg_Log_Chr_only.pdf")
plt.show()

<h3 style="text-align:center; color:orange;"> - REGRESSION OF BMI AGAINST CHRISTENSENELLACEAE ABUNDANCE - </h3>
<h4 style="text-align:center; color:orange;"> - These are regressions including individuals WITH & WITHOUT Christensenellaceae - </h4>

In [ ]:
# Get log10 transformation of counts
mapChrist["ChristLog"] = np.log10(mapChrist["Christ"])
# Get arcsin sqrt transformation of counts (see Structure, Function, and Diversity of the Healthy Human Microbiome)
mapChrist["ChristArcSin"] = np.arcsin(np.sqrt(mapChrist["Christ"]))

# Set Colors
colors = {'African American':'red', 'Asian or Pacific Islander':'blue', 'Caucasian':'green', 'Hispanic':'yellow'}

# Remove samples w/ no Christ
#mapChrist = mapChrist[mapChrist["Christ"] != 0]

# Plot ArcSin
mapChrist.plot(kind='scatter', x='bmi', y='ChristArcSin', c=mapChrist['race'].apply(lambda x: colors[x]), figsize=[10,10])
plt.title("ArcSin sqrt Transformation")

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(mapChrist["bmi"],mapChrist["ChristArcSin"])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'r--')
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/reg_ArcSin_All.pdf")
plt.show()

# Plot Log10
mapChrist.plot(kind='scatter', x='bmi', y='ChristLog', c=mapChrist['race'].apply(lambda x: colors[x]), figsize=[10,10])
plt.title("Log10 Transformation")

mapChrist["ChristLog"] = mapChrist["ChristLog"].replace(to_replace='-inf', value=-20)

slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(mapChrist["bmi"],mapChrist["ChristLog"])
line = slope*mapChrist["bmi"]+intercept
plt.plot(mapChrist["bmi"],line, 'r--')
print "Slope: " + str(slope)
print "R^2  : " + str(r_value*r_value)
print "p-val: " + str(p_value)
plt.savefig("/Users/brooks/Dropbox/American_Gut/Figures/S5_HMP_Overlap/reg_Log_All_-20.pdf")
plt.show()

<h3 style="text-align:center; color:orange;"> - REGRESSION OF BMI AGAINST CHRISTENSENELLACEAE ABUNDANCE SUBSET BY RACE - NOT USED IN MANUSCRIPT - </h3>
<h4 style="text-align:center; color:orange;"> - These are regressions only including individuals with Christensenellaceae - </h4>

In [ ]:
import scipy

# Get log10 transformation of counts
mapChrist["ChristLog"] = np.log10(mapChrist["Christ"])
# Get arcsin sqrt transformation of counts (see Structure, Function, and Diversity of the Healthy Human Microbiome)
mapChrist["ChristArcSin"] = np.arcsin(np.sqrt(mapChrist["Christ"]))


def plotChrist(mapTableChrist, raceIn, colorPattern):
    mapTableChrist = mapChrist[mapChrist['race'] == raceIn]
    
    # Exclude Individuals w/o Christ
    mapTableChrist = mapTableChrist[mapTableChrist["Christ"] != 0]
    
    #print mapTableChrist["ChristArcSin"]
    mapTableChrist.plot(kind='scatter', x='bmi', y='ChristArcSin', figsize=[10,10])
    
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(mapTableChrist["bmi"],mapTableChrist["ChristArcSin"])

    line = slope*mapTableChrist["bmi"]+intercept
    plt.plot(mapTableChrist["bmi"],line, colorPattern)
    plt.title(raceIn, size=30)
    
    print
    print raceIn
    print "Slope: " + str(slope)
    print "R^2  : " + str(r_value*r_value)
    print "p-val: " + str(p_value)
    plt.show()

plotChrist(mapChrist, "African American", 'r--')
plotChrist(mapChrist, "Asian or Pacific Islander", 'r--')
plotChrist(mapChrist, "Caucasian", 'r--')
plotChrist(mapChrist, "Hispanic", 'r--')

<h3 style="text-align:center; color:orange;"> - REGRESSION OF BMI AGAINST CHRISTENSENELLACEAE ABUNDANCE SUBSET BY RACE - NOT USED IN MANUSCRIPT - </h3>
<h4 style="text-align:center; color:orange;"> - These are regressions including individuals WITH & WITHOUT Christensenellaceae - </h4>

In [ ]:
import scipy


# Get log10 transformation of counts
mapChrist["ChristLog"] = np.log10(mapChrist["Christ"])
# Get arcsin sqrt transformation of counts (see Structure, Function, and Diversity of the Healthy Human Microbiome)
mapChrist["ChristArcSin"] = np.arcsin(np.sqrt(mapChrist["Christ"]))


def plotChrist(mapTableChrist, raceIn, colorPattern):
    mapTableChrist = mapChrist[mapChrist['race'] == raceIn]
    
    # Exclude Individuals w/o Christ
    mapTableChrist = mapTableChrist[mapTableChrist["Christ"] != 0]
    
    # Exclude outliers w/ bmi > 35
    #mapTableChrist = mapTableChrist[mapTableChrist["bmi"]  < 35]
    
    # Include Individuals w/o Christ
    #mapTableChrist["ChristLog"] = mapTableChrist["ChristLog"].replace(to_replace='-inf', value=-10)
    
    
    #print mapTableChrist["ChristLog"]
    mapTableChrist.plot(kind='scatter', x='bmi', y='ChristLog', figsize=[10,10])
    
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(mapTableChrist["bmi"],mapTableChrist["ChristLog"])

    line = slope*mapTableChrist["bmi"]+intercept
    plt.plot(mapTableChrist["bmi"],line, colorPattern)
    plt.title(raceIn, size=30)
    
    print
    print raceIn
    print "Slope: " + str(slope)
    print "R^2  : " + str(r_value*r_value)
    print "p-val: " + str(p_value)
    plt.show()

plotChrist(mapChrist, "African American", 'r--')
plotChrist(mapChrist, "Asian or Pacific Islander", 'r--')
plotChrist(mapChrist, "Caucasian", 'r--')
plotChrist(mapChrist, "Hispanic", 'r--')

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h1 style="text-align:center; color:orange;"> - Additional Imports - </h1>
<h3 style="text-align:center; color:blue;"> - Rarefied Tables - </h3>

In [ ]:
# PATH TO RAREFIED TABLES #
tablesFP = "23_0_multiple_rarefactions_1000"

def import_rare_tables(rareFolderIn):
    print " - Importing Rarefied Tables - "
    # ARRAY TO STORE ALL RAREFIED TABLES #
    rareTablesIn = []
    # DICTIONARY (SAMPLE) OF DICTIONARY (OTUS) OF ARRAY (EACH TABLES COUNT) #
    countsDict = {}
    
    # LOOP THROUGH ALL TABLES #
    for idx, i in enumerate(glob.glob(rareFolderIn + "/*")):
        
        if idx % 10 == 0: print idx
        # INPUT RAREFIED TABLE #
        rareTableIn = load_table(i)
        # STORE RAREFIED TABLE #
        rareTablesIn.append(rareTableIn)
        
        # GET NON-ZERO LOCATIONS [[otuLocs], [sampleLocs]] #
        nnzLocs = rareTableIn.matrix_data.nonzero()
        # GET SAMPLES #
        samplesRare = rareTableIn.ids(axis='sample')
        # GET OTUS #
        otusRare = rareTableIn.ids(axis='observation')
        
        # INITIATE DICTIONARY SAMPLES #
        if idx == 0:
            for j in samplesRare: countsDict[j] = {}

        # FOR EACH NON-ZERO LOCATION #
        for k in np.arange(len(nnzLocs[0])):
            
            # CHECK IF OTU IS IN SAMPLE'S DICTIONARY AND IF NOT INITIATE AS ARRAY #
            if otusRare[nnzLocs[0][k]] not in countsDict[samplesRare[nnzLocs[1][k]]]: countsDict[samplesRare[nnzLocs[1][k]]][otusRare[nnzLocs[0][k]]] = []
            
            # ADD NON-ZERO VALUE TO THE ARRAY FOR THE SPECIFIC SAMPLE AND OTU
            countsDict[samplesRare[nnzLocs[1][k]]][otusRare[nnzLocs[0][k]]].append(rareTableIn.get_value_by_ids(otusRare[nnzLocs[0][k]], samplesRare[nnzLocs[1][k]]))
        
        
    return rareTablesIn, countsDict

rareTables, rareCounts = import_rare_tables(tablesFP)

<h3 style="text-align:center; color:blue;"> - Beta Diversity - </h3>

In [ ]:
# PATH TO BETA DIVERSITY DISTANCE MATRICES #
betasFP = "24_0_beta_diversity_1000_bray_curtis"
# CREATE EMPTY LIST TO STORE DATA FRAMES #
betasDFs = []; 
# FOR EACH DATAFRAME... GO THROUGH AND IMPORT #
print " - Importing Dataframes - "
for idx, i in enumerate(glob.glob(betasFP + "/*")):
    #print idx, i
    betasDFs.append(pd.read_csv(i, sep='\t', index_col=0, skiprows=None, verbose=False))

# COLLATE THE DATAFRAMES #
print " - Collating Dataframes - "
betasCollate = pd.concat(betasDFs[:]) 

# FIND AND WRITE MEAN DATAFRAME #
print " - Calculating and Saving Mean - "
betasMean = betasCollate.groupby(betasCollate.index).mean()
#pandasMean.to_csv(dirPath + "mean_distances.txt", sep="\t")

# FIND AND WRITE STANDARD DEVIATION DATAFRAME #
print " - Calculating and Saving Standard Deviation - "
betasStd = betasCollate.groupby(betasCollate.index).std()
#pandasStd.to_csv(dirPath + "std_distances.txt", sep="\t")

<h3 style="text-align:center; color:blue;"> - PCoA - </h3>

In [ ]:
pcoasFP = "24_3_principle_coordinates_1000_bray_curtis/"

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h1 style="text-align:center; color:orange;"> - Additional Imports - </h1>
<h3 style="text-align:center; color:blue;"> - Rarefied Tables - </h3>

In [ ]:
# FILTER OTUS WITH LESS THAN 10 COUNTS #
filterTable = filter_otu_mincount(biomTable, 10)
get_table_info(filterTable)

# GET TABLE AS RELATIVE ABUNDANCE #
relativeTable = get_table_relative(filterTable)

# GET 2D COUNT ARRAY OF RELAITVE TABLE #
relativeCountArray = count_array(relativeTable)

# CONVERT 2D ARRAY TO MATRIX #
rMat = np.matrix(relativeCountArray)

# GET OTU CORRELATION MATRIX #
otuCorr = np.corrcoef(rMat.T)

In [ ]:
#np.random.seed(51990) # random seed for consistency
%matplotlib inline
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


def pca_2d(dataIn, figBool=False):
    
    # ESTABLISH PCA MODEL AND NUMBER OF COMPONENTS #
    pca = PCA(n_components=2)
    pcaVec = pca.fit_transform(dataIn)

    if figBool == True:
        plt.figure(figsize=[10,10])
        plt.scatter(pcaVec[:, 0], pcaVec[:, 1], alpha=.8, lw=2)
        plt.title('PCA')
        plt.xlabel("PC1: " + str(pca.explained_variance_ratio_[0]))
        plt.ylabel("PC2: " + str(pca.explained_variance_ratio_[1]))
        plt.show()
    else: 
        print("Explained variance of components: " + str(pca.explained_variance_ratio_))
    
    return pcaVec
    
pca_2d(otuCorr, figBool=True)

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h1 style="text-align:center; color:orange;"> - Beta Diversity - </h1>
<h2 style="text-align:center; color:blue;"> - Calculate Beta Diversity - </h2>

In [ ]:
### MANTEL CORRELATION OF DISTANCE MATRICES ###
from skbio.stats.distance import mantel
def mantel(dm1In, dm2In):
    print " - Performing Mantel Test - "
    rMantel, pMantel, dimMantel = mantel(dm1In, dm2In)
    print "   Mantel R-value   | ",rMantel
    print "   Mantel P-value   | ",pMantel
    print "   Mantel Dimensions| ",dimMantel
    return rMantel, pMantel, dimMantel
mantel(dmBrayCurtis, dmJaccard)

In [ ]:
### ANOSIM ASSOCIATION OF DISTANCE MATRIX AND METADATA CATEGORY ###
from skbio.stats.distance import ANOSIM
print " - Perform Anosim Test - "
anosim = ANOSIM(dmBraycurtis, mapDf, column='race')
results = anosim(999)
print "   Test Statistic| ",results.statistic
print "   P-Value       | ",results.p_value
print "   Permutations  | ",results.permutations
print "   Groups        | ",results.groups
print "   Sample Size   | ",results.sample_size

In [ ]:
from skbio.stats.distance import PERMANOVA
print " - Perform Permanova Test - "
permanovaResults = PERMANOVA(dmBraycurtis, mapDf, column='race')
results = permanovaResults(9999)
print "   Test Statistic| ",results.statistic
print "   P-Value       | ",results.p_value
print "   Permutations  | ",results.permutations
print "   Groups        | ",results.groups
print "   Sample Size   | ",results.sample_size

In [ ]:
from skbio.stats.distance import bioenv
print " - Perform BioEnv Test - "
print "   (Correlation b/t Distance Matrix and Continuous Variable)"
bioenvResults = bioenv(dmBraycurtis, mapDf, columns=['age_years', 'bmi'])
print bioenvResults

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h1 style="text-align:center; color:orange;"> - Custom Pipelines - </h1>
<h3 style="text-align:center; color:red;"> - Generate Taxonomic Bar Chart - </h3>

<h3 style="text-align:center; color:red;"> - OTU Abundance Cutoff Analysis - </h3>
<h5 style="text-align:left; color:black;">Filter OTUs by minimum abundance cutoffs and write tables</h5>

In [ ]:
### BIOM ABUNDANT ### - filter the otu table to different min_counts and analyze
def biom_abundant(bt, out_fp):
    # MIN_COUNTS TO FILTER BY
    min_nums = [2, 3, 4, 5, 10, 20, 30, 40, 50, 75, 100, 250, 500, 750, 1000, 2500, 5000, 10000]
    # GENERATE ABUNDANT FOLDER
    abun_folder = dirPath+'abundant_partition/'
    if not os.path.isdir(abun_folder): os.makedirs(abun_folder)
    # LOOP THROUGH MIN_COUNTS
    for min_num in min_nums:        
        # FILTER BIOM TABLE
        ft = filter_otu_mincount(bt, min_num)
        # WRITE THE FILTERED TABLE
        write_table(ft, abun_folder + str(min_num) + '_filtered_table.txt')
        # GET GENERAL INFO
        get_table_info(ft, writePath=dirPath+'abundant_partition/'+str(min_num)+"_summary_filtered_otus.txt", printOut=False)

biom_abundant(biomTable, dirPath)

<h3 style="text-align:center; color:red;"> - Filter Map to Samples in Table - </h3>
<h5 style="text-align:left; color:black;">Filter a mapping file to </h5>

<h1 style="text-align:center; color:black;">------------------------------------------------------------------------------</h1>
<h1 style="text-align:center; color:orange;"> - In Progress - </h1>
<h3 style="text-align:center; color:red;"> - Perform Rarefactions - </h3>

In [ ]:
def rarefaction(bt, rarefactionCutoff=None, seed=200):
    prng = np.random.RandomState(seed) # Set random seed for reproducible results
    if rarefactionCutoff == None: rarefactionCutoff = min(bt.sum('sample')) # If no rarefaction cutoff provided = minimum sample sum of counts
    iterSamples = bt.copy() # Make a copy of the table to store rarefied values
    for idx, (values, id, metadata) in enumerate(iterSamples): # for each sample
        probability_measured = values / float(sum(values)) # probability of sampling = sample relative frequency
        choice = prng.choice(len(bt.ids(axis='observation')), rarefactionCutoff, p=probability_measured)
        print idx, choice
""" 
    trarefied = np.empty_like(bt)
    for i in range(bt.shape[0]): # for each sample
        
        p = bt[i] / float(sample_counts_rarefaction[i]) # relative frequency / probability
        choice = prng.choice(number_of_observations, rarefactionCutoff, p=p)
        trarefied[i] = np.bincount(choice, minlength=nvar)

    return trarefied"""
rarefaction(biomTable)